In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


## BOH3 and DDS4 Qualtrics

In [ ]:
workbookpath = '2025\BOH3+DDS4\Session 1\\2025 BOH3 and DDS4_0.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath, keep_default_na=False, na_values=[''])  # This will keep 'NA' text as it is
# turn evey column to lowercase
df.columns = df.columns.str.lower()
# strip all columns
df.columns = df.columns.str.strip()
# remove non-ascii characters
df.columns = df.columns.str.replace(r'[^\x00-\x7F]+', '', regex=True)
df.loc[0] = df.loc[0].str.replace(r'[^\x00-\x7F]+', '', regex=True)
supervisorcol = [col for col in df.columns if 'evaluation#2' in col.lower()]
studentcol = [col for col in df.columns if 'evaluation#1' in col.lower()]
display(df[supervisorcol[3]].value_counts(dropna=False))
fullnamedict = {col: df.loc[0, col] for col in df.columns}
row0 = df.loc[0]
# row0 = row0.str.replace(r'[^\x00-\x7F]+', '', regex=True)
df.drop(0, inplace=True)
colAge = [col for col in df.columns if '#2_1_1' in col]
colPtInfo = [col for col in df.columns if re.match(r'pt\.\s*\d+', col)]
def getAgeGroup(age):
    if age <= 6:
        return '0-6'
    elif age <= 17:
        return '7-17'
    elif age >= 18 and age<140:
        return '18+'
    else:
        return np.nan
    
for col in colAge:
    df[col] = df[col].astype(float)
colAgeNew = []
for col in colAge:
    df[f'{col} group'] = df[col].apply(getAgeGroup)
    colAgeNew.append(f'{col} group')
colCode = [col for col in df.columns if 'code' in col.lower()]
colCodeNew = []
for col in colCode:
    df[col] = df[col].astype(str)
    df[f'{col} list'] = df[col].apply(lambda x: [f'0{item}' if item.isdigit() and len(item) == 2 else item for item in re.split(r',\s*', x)])  # Handles optional spaces after commas and adds 0 before two-digit items
    colCodeNew.append(f'{col} list')
# display(df)

# print(colAge)
# display(df.loc[0, colAge])
colCohort = 'cohort'
dfboh3 = df[df[colCohort]=='BOH3']
dfdds4 = df[df[colCohort]=='DDS4']
# display(dfboh3.head())

colNPts = 'No. of pts'
colId = 'student id'
colOprStuFeedback = 'student reflection'
colCI = 'Clinical Incident'
colCIExp = 'CI Explanation'
colPriority = [col for col in df.columns if 'priority' in col.lower()]
# patient info columns are with Pt.\d+ in the column name

print(colPtInfo)
importantCols =  ['SO Feedback',	'SO Edu Feedback',	'SO Edu Name', 'Obs Feedback',	'Obs Edu Feedback', 'Obs Edu Name']
columnList = [
    "Student Reflection",
    "Edu Feedback",
    "Entrustment",
    "Clinical Incident",
    "CI Explanation",
    "Edu Name"
]
importantCols.extend(columnList)
print(fullnamedict)
# dfboh3[dfboh3[colId]==1472946]

### Create age report

In [ ]:
def getNPatients(text):
    # print(text)
    if pd.isna(text):
        return 0
    # select the number of patients from the text, select the highest number Patient 1, Patient 2, Patient 3 etc.
    numbers = re.findall(r'\d+', text)
    if len(numbers) == 0 and 'fta' in text.lower():
        return 'Patient FTA'
    # get the highest number
    if len(numbers):
        return max([int(num) for num in numbers])
    else:
        return 0
    
def createGeneralReport(df, cohort, savefolder):
    doc = SimpleDocTemplate(f'{savefolder}\\{cohort} Summary.pdf', pagesize=pageSize, leftMargin=leftMargin,
                            rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    idCounts = df[colId].value_counts()
    ids = idCounts.index
    elements = []
    patientCountDf = pd.DataFrame(columns=['Student ID', 'Student Name', '# Patients', 'Patient Ages',
                                           '#18+', '#7-17', '#0-6', 'Clinic'])
    evalDf = pd.DataFrame(columns=['Student ID', 'Student Name', '# Yes', '# No', '# N/A', " No%"])
    
    for id in ids:
        studentDf = df[df[colId] == id]
        studentName = studentDf['given name'].values[0] + ' ' + studentDf['family name'].values[0]
        studentDf['Patient Count'] = studentDf[colNPts.lower()].apply(getNPatients)
        studentDf['Patient Count'] = studentDf['Patient Count'].replace('Patient FTA', 0).astype(int)
        patientCount = studentDf['Patient Count'].sum()
        ageList = []
        for col in colAge:
            studentDf[col] = studentDf[col].astype(float)
            ageList.extend(studentDf[col].dropna().to_list())
        ageList.sort()
        n18plus = len([age for age in ageList if age >= 18])
        n7to17 = len([age for age in ageList if age >= 7 and age < 18])
        n0to6 = len([age for age in ageList if age >= 0 and age < 7])
        clinic = studentDf['clinic'].values[0]
        patientCountDf = pd.concat([patientCountDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName], '# Patients': [patientCount],
             "Patient Ages": [ageList], '#18+': [n18plus], '#7-17': [n7to17], '#0-6': [n0to6], 'Clinic': [clinic]})])
    
        # get the evaluation counts for supervisorcol
        nYes = 0
        nNo = 0
        nNA = 0
        
        for col in supervisorcol:
            nYes += studentDf[col].str.contains('Yes').sum()
            nNo += studentDf[col].str.contains('No').sum()
            nNA += studentDf[col].str.contains('NA').sum()
        nopercent = nNo / (nYes + nNo + nNA + 0.001) * 100
        evalDf = pd.concat([evalDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName],
                                                  '# Yes': [nYes], '# No': [nNo], '# N/A': [nNA], ' No%': [nopercent]})])
    
    evalDf.sort_values(by=[' No%', '# Yes'], ascending=[True, False], inplace=True)
    
    # Create a stacked plot of evaluation counts with student id as x axis
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]))
    evalDf.set_index('Student ID')[['# Yes', '# No', '# N/A']].iloc[::-1].plot(kind='barh', stacked=True, ax=ax, width=0.8)
    ax.set_xlabel('Count')
    ax.set_ylabel('Student ID')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=8)
    plt.subplots_adjust(hspace=0.5)
    plt.tight_layout()
    img = addPlotImage(fig, 0.9)
    
    elements.append(Paragraph(f'Performance Overview for {cohort}', subsubheadingStyle))
    elements.append(Spacer(1, 12))
    elements.append(img)
    elements.append(Spacer(1, 6))
    elements.append(Paragraph("The students at the bottom are with highest % of No Responses", tableTextStyle))
    
    # Create a table for patientCountDf with bar graphs in the Patient Ages column
    table_data = [['Student ID', 'Student Name', '# Patients', 'Patient Ages', 'Age count', 'Clinic']]
    patientCountDf.sort_values(by='# Patients', ascending=False, inplace=True)
    

    unique_clinics = patientCountDf['Clinic'].unique()
    print(unique_clinics)
    clinic_colors = {clinic: color for clinic, color in zip(unique_clinics, sns.color_palette("husl", len(unique_clinics)))}
    
    # Create a legend for clinic colors
    legend_data = [[Paragraph('Clinic', tableTextStyle), Paragraph('Color', tableTextStyle)]]
    for clinic, color in clinic_colors.items():
        legend_data.append([Paragraph(clinic, tableTextStyle), ''])
    
    legend_table = Table(legend_data, colWidths=[2 * inch, 1 * inch])
    legend_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 12),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    
    for i, (clinic, color) in enumerate(clinic_colors.items(), start=1):
        legend_table.setStyle(TableStyle([
            ('BACKGROUND', (1, i), (1, i), color)
        ]))
    

    
    for i, row in patientCountDf.iterrows():
        # Create a tiny bar graph for the Patient Ages column
        fig, ax = plt.subplots(figsize=(2, 0.5))
        sns.histplot(row['Patient Ages'], bins=[i for i in range(0, 100, 6)], ax=ax)
        ax.yaxis.set_visible(False)
        ax.xaxis.set_visible(False)
        ax.axvline(x=6.5, color='black', linestyle='dotted')
        ax.axvline(x=18, color='black', linestyle='dotted')
        plt.tight_layout()
        imgdata = io.BytesIO()
        fig.savefig(imgdata, format='png', bbox_inches='tight')
        plt.close(fig)
        imgdata.seek(0)
        img = Image(imgdata, width=4 * inch, height=1 * inch)
        agedict = {'18+': row['#18+'], '7-17': row['#7-17'], '0-6': row['#0-6']}
        agetext = '<br/>'.join([f'{key}: {value}' for key, value in agedict.items()])
        table_data.append([row['Student ID'], Paragraph(row['Student Name'], tableTextStyle),
                           row['# Patients'], img, Paragraph(agetext, tableTextStyle), Paragraph(row['Clinic'], tableTextStyle)])
        
    table = Table(table_data, colWidths=[1.2 * inch, 1.5 * inch, 1 * inch, 4 * inch, 1.2 * inch, 1.2 * inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 0.2 * inch),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
        ('BOTTOMPADDING', (3, 1), (3, -1), 0),  
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    
    # for i, row in enumerate(patientCountDf.iterrows(), start=1):
    #     clinic = row[1]['Clinic']
    #     if clinic in clinic_colors:
    #         table.setStyle(TableStyle([
    #             ('BACKGROUND', (0, i), (-1, i), clinic_colors[clinic])
    #         ]))
    elements.append(PageBreak())
    elements.append(Paragraph(f'Patient Count and Age Distribution for {cohort}', subsubheadingStyle))
    elements.append(Spacer(1, 12))
    text = f"""
The table shows ages of patients with patient count and age group distribution, but all information
                              regarding patient ages was not filled appropriately. Thus, the age distribution is not accurate for
                              all students.The dotted lines in the graphs indicate the age groups 0-6 and 7-17.
    """
    text2 = f"""Average Patient count is {round(patientCountDf['# Patients'].mean(), -1)} per student. """
    text3 = f"""The highest number of patients seen by a student is {patientCountDf['# Patients'].max()}."""
    text4 = f"""The lowest number of patients seen by a student is {patientCountDf['# Patients'].min()}.""" 
    text5 = "Data is not very accurate due to missing fields in the forms but should be close."
    elements.append(Paragraph(text, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text2, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text3, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text4, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text5, tableTextStyle))
    elements.append(Spacer(1, 12))

    # elements.append(Paragraph('Clinic Colors Legend', subsubheadingStyle))
    # elements.append(Spacer(1, 12))
    # elements.append(legend_table)
    # elements.append(Spacer(1, 12))

    elements.append(table)
    
    # doc.build(elements)
    # display(patientCountDf)
    # display(evalDf)
    return elements
# savefolder = f'{folder}'
# createGeneralReport(dfboh3, 'BOH3', savefolder)

In [ ]:
def createfronttable(df: pd.DataFrame, title = 'Front Page Summary', type_=None):
    nforms = len(df)
    # age range  dictionary
    ageDict = df[colAgeNew].apply(pd.Series.value_counts).to_dict()
    result = {}
    for group in ageDict.values():
        for key, value in group.items():
            result[key] = result.get(key, 0) + (value if not np.isnan(value) else 0)
    ageDict = result
    print(ageDict)
    # total patients
    df['num patients'] = df[colNPts.lower()].apply(getNPatients) 
    df['num patients'] = df['num patients'].replace('Patient FTA', 0).astype(int)
    # get total patients seen by each student
    studentpatients = df.groupby(colId)['num patients'].sum().to_dict()
    print(studentpatients)
    # filter out max and min vlaues
    maxPatients = max(studentpatients.values())
    minPatients = min(studentpatients.values())
    print(minPatients, maxPatients)
    # get num patients for each clinic
    clinicPatients = df.groupby('clinic')['num patients'].sum().to_dict()
    clinicPatietnsText = "<br/>".join([f"{key}: {value}" for key, value in clinicPatients.items()])
    totalPatients = df['num patients'].sum()
    print(totalPatients)
    # average patient per student
    dfoperators = df[df[colRole.lower()] == 'Operator']
    noperators = len(dfoperators)
    noperatorstudents = len(dfoperators[colId].unique())
    nstudents = len(df[colId].unique())
    avgPatients = totalPatients / noperatorstudents
    # average patient per form
    avgPatientsForm = totalPatients / noperators
    print(avgPatients)
    print(avgPatientsForm)
    # priority counts
    priorityDict = df[colPriority].apply(pd.Series.value_counts).to_dict()
    result = {}
    for group in priorityDict.values():
        for key, value in group.items():
            # if , then split the key and add the value to the key
            if ',' in key:
                keys = key.split(',')
                for key in keys:
                    result[key] = result.get(key, 0) + (value if not np.isnan(value) else 0)
            else:
                result[key] = result.get(key, 0) + (value if not np.isnan(value) else 0)
    priorityDict = result
    print(priorityDict)
    # item codes count
    itemCodeList = []
    for col in colCodeNew:
        itemCodeList.extend(df[col].to_list())
    itemCodeList = [item for sublist in itemCodeList for item in sublist]
    itemCodeDict = {item: itemCodeList.count(item) for item in set(itemCodeList)}
    # remove nan
    itemCodeDict.pop('nan', None)
    print(itemCodeDict)
    # get top 5 item codes
    top5ItemCodes = dict(sorted(itemCodeDict.items(), key=lambda x: x[1], reverse=True)[:5])
    print(top5ItemCodes)
    # entrustment counts
    entrustmentCounts = df['entrustment'].value_counts().to_dict()
    # split the keys with :
    entrustmentCounts = {key.split(':')[0]: value for key, value in entrustmentCounts.items()}
    print(entrustmentCounts)
    # extract number Lvl \d+ from entrustment
    df['entrustment lvl'] = df['entrustment'].astype(str).str.extract(r'Lvl (\d+)').fillna(0)
    df['entrustment lvl'] = df['entrustment lvl'].astype(int)
    # yes/no/na counts
    df['nYesStudent'] = df[studentcol].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['nNoStudent'] = df[studentcol].apply(lambda x: (x == 'No').sum(), axis=1)
    df['nNAStudent'] = df[studentcol].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['scoreStu'] = df['nYesStudent']/(df['nYesStudent'] + df['nNoStudent'] + 0.0001)
    df['nYesSupervisor'] = df[supervisorcol].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['nNoSupervisor'] = df[supervisorcol].apply(lambda x: (x == 'No').sum(), axis=1)
    df['nNASupervisor'] = df[supervisorcol].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['scoreSup'] = df['nYesSupervisor']/(df['nYesSupervisor'] + df['nNoSupervisor'] + 0.0001)
    df['scorediff'] = df['scoreStu'] - df['scoreSup']
    scorediffavg = df['scorediff'].mean()
    # filter out df with entrustment non zero
    dfentrustment = df[df['entrustment lvl'] > 0]
    dfentrustment.to_excel(f'{folder}\\entrustment.xlsx')
    # correlation between entrustment and score supervisor
    entrustmentCorr = dfentrustment['entrustment lvl'].corr(dfentrustment['scoreSup'])
    print(entrustmentCorr)
    # get column wise Yes/No/NA counts for supervisorcols
    dfcolumnwise = pd.DataFrame(index=supervisorcol, columns=['key', 'Question','Yes', 'No', 'NA'])
    for col in supervisorcol:
        dfcolumnwise.loc[col, 'Yes'] = df[col].str.contains('Yes').sum()
        dfcolumnwise.loc[col, 'No'] = df[col].str.contains('No').sum()
        dfcolumnwise.loc[col, 'NA'] = df[col].str.contains('NA').sum()
        dfcolumnwise.loc[col, 'Question'] = fullnamedict[col].split(' - ')[-1].split('\n')[-1]
        dfcolumnwise.loc[col, 'key'] = fullnamedict[col].split(' - ')[-1].split('\n')[0]
    dfcolumnwise['score'] = dfcolumnwise['Yes']/(dfcolumnwise['Yes'] + dfcolumnwise['No'] + 0.0001)
    dfcolumnwise.sort_values(by='score', ascending=False, inplace=True)
    bestQuestion = dfcolumnwise.loc[dfcolumnwise.index[0], 'key']
    best3Questions = dfcolumnwise.loc[dfcolumnwise.index[:2], 'key']

    worstQuestion = dfcolumnwise.loc[dfcolumnwise.index[-1], 'key']
    worst3Questions = dfcolumnwise.loc[dfcolumnwise.index[-3:], 'key']

    # get role wise counts
    dfrole = df[colRole.lower()].value_counts().to_dict()
    roltext = "<br/>".join([f"{key}: {value}" for key, value in dfrole.items()])
    # display(dfcolumnwise)
    
    ageOrder = ["0-6", "7-17", "18+"]

    # Convert ageDict to formatted text in the specified order
    agetext = "<br/>".join([f"{key}: {int(ageDict[key])}" for key in ageOrder if key in ageDict])
    prioritytext = "<br/>".join([f"{key.split('Group ')[-1]}: {int(priorityDict[key])}" for key in priorityDict])
    # create a table for the front page
    dfFront = pd.DataFrame([
    ['Total number of forms submitted', nforms],
    ['Role Distribution', roltext],
    ['Age Range Counts', agetext],
    ['Total number of patients managed by students', totalPatients],
    ['Average number of patients managed per student (min - max)', f'{int(avgPatients)}   ({minPatients} - {maxPatients})'],
    ['Number of forms receiving entrustment level 3 or 4', (df['entrustment lvl'] >= 3).sum()],
    ['Number of forms receiving entrustment level 2', (df['entrustment lvl'] == 2).sum()],
    ['Number of forms receiving entrustment level 1', (df['entrustment lvl'] == 1).sum()],
    ['Correlation between entrustment and supervisor score(0-1)<br/>(Higher correlation implies supervisor are giving entrustment scores based on objective checklist scores and have less latent bias, should be 0.2+ <br/>*Not valid for small number of students)', round(entrustmentCorr, 2)],
    ['Difference in student and supervisor ratings (avg)', round(scorediffavg,3)],
    ['Top 5 Item Codes', ', '.join(top5ItemCodes.keys())],  
    ['Priority Group', prioritytext],
    ['Areas students are performing well in', ", ".join(best3Questions)],
    ['Areas that need improvement', ", ".join(worst3Questions)],

    ['Clinic Patient Counts', clinicPatietnsText]
], columns=['Metric', 'Value'])
    if type_ == 'clinic': # remove last row
        dfFront = dfFront.iloc[:-1]
    # # Adding Age Range
    # for key, value in ageDict.items():
    #     dfFront.loc[len(dfFront)] = [f'Age range {key}', value]

    # Adding Priority Group Counts
    # for key, value in priorityDict.items():
    #     dfFront.loc[len(dfFront)] = [f'Priority Group {key}', int(value)]

    # Adding Top 5 Item Codes
    # for key, value in top5ItemCodes.items():
    #     dfFront.loc[len(dfFront)] = [f'Item Code {key}', value]
    # display(dfFront)
    dfFrontTable = createTable(dfFront, title, colRatio=[3,2], customTextCols=[0, 1], tableTextStyle=tableTextStyleSmall)
    # table is 2nd last item of keepTogether dfFrontTable
    table = dfFrontTable._content[2]
    # change top and bottom padding of the table
    padding = 12
    if type_ == 'clinic':
        padding = 6
    table.setStyle(TableStyle([
        ('TOPPADDING', (0, 0), (-1, -1), padding),
        ('BOTTOMPADDING', (0, 0), (-1, -1), padding),
    ]))
    # get all the contents of dfFrontTable
    # for content in dfFrontTable._content:
    #     print(content)
    # change text of 3rd row and 2nd column to age paragraph
    numRows = len(table._cellvalues)  # Number of rows
    numCols = len(table._cellvalues[0]) if numRows > 0 else 0  # Number of columns

    print(f"Rows: {numRows}, Columns: {numCols}")
    # table._cellvalues[2][1] = ageparagraph
    return dfFrontTable
createfronttable(df)

In [ ]:

# get the ages from all the columns
def plotages(df, title):
    ages = []
    for col in colAge:
        ages.extend(df[col].to_list())
    
    # remove nan
    ages = [age for age in ages if not np.isnan(age)]
    print(ages)
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]*0.8))
    sns.histplot(ages, bins=[i for i in range(0, 100, 10)], ax=ax)
    # show total number of ages
    total = len(ages)
    plt.text(0.90, 0.90, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    plt.title(title)
    plt.xlabel('Age')
    return fig

def plotpie(df, title):
    # plot pie chart of counts of Role column
    fig, ax = plt.subplots(figsize=figSize)
    total = len(df)
    df[colRole.lower()].value_counts().plot.pie(ax=ax, autopct=lambda x: autopct(x, total))
    plt.title(title)
    plt.text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    return fig



def plotagegroups(df, title):
    dummydf = pd.DataFrame(columns = ['Age Group', 'Code'])
    for ageCol, codeCol in zip(colAgeNew, colCodeNew):
        dummydf = pd.concat([dummydf, df[[ageCol, codeCol]].rename(columns={ageCol: 'Age Group', codeCol: 'Code'})])
    # remove nan ages
    dummydf = dummydf[dummydf['Age Group'].notna()]
    print(dummydf['Age Group'].unique())
    # display(dummydf.head(20))
    nsubplots = len(dummydf['Age Group'].unique())
    # create a square grid of subplots
    rows = int(np.ceil(nsubplots/2))
    cols = 2
    fig, ax = plt.subplots(nsubplots, 1, figsize=(figSize[0], figSize[1]))    
    # ax = ax.flatten()
    j=0
    for i, group in enumerate(dummydf['Age Group'].unique()):
        # remove nan entries
        print(f'{group}: {dummydf[dummydf["Age Group"]==group].shape[0]}')
        total = dummydf[dummydf['Age Group']==group].shape[0]
        # get all the codes
        codes = []
        for j, row in dummydf[dummydf['Age Group']==group].iterrows():
            codes.extend(row['Code'])
        # remove nan
        codes = [code for code in codes if code != 'nan']
        # remove if not a number
        # codes = [code for code in codes if code.isnumeric()]
        # remove if more than 5 characters
        codes = [code for code in codes if len(code) <= 5]
        print(codes)
        # Calculate frequency of each code
        code_counts = pd.Series(codes).value_counts(ascending=False)
        # remove counts less than 2
        if len(code_counts)>15:
            code_counts = code_counts[code_counts>=3]
        # take top 15
        code_counts = code_counts[:15]
        sns.barplot(x=code_counts.index, y=code_counts.values, ax=ax[i])
        ax[i].set_title(f'({group} years)')
        ax[i].set_xlabel('Items')
        ax[i].set_ylabel('Frequency')
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=90, fontsize=8)
        ax[i].text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax[i].transAxes)
        # y ticks should be integers
        ax[i].yaxis.get_major_locator().set_params(integer=True)
    # remove the last subplot if not needed
    # if nsubplots % 2:
    #     fig.delaxes(ax[-1])
    # plt.title(f'{title} - {group}')
    # plt.xlabel('Codes')
    # increase the space between subplots
    plt.tight_layout()
    plt.suptitle(f'{title} - #Items performed by Age Group (Top 15)', fontsize=16, y=1.02)
    return fig

def createcohortreport(df, cohort):

    doc = SimpleDocTemplate(f'{folder}\\{cohort} session 1 summary.pdf', pagesize= pageSize, leftMargin = leftMargin,
                                rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    elements = []
    elements.append(Paragraph(f'{cohort} Summary', headingStyle))
    elements.append(Spacer(1, 24))
    frontpagetable = createfronttable(df)
    elements.append(frontpagetable)
    elements.append(PageBreak())
    # elements.append(Spacer(1, 24))
    fig1 = plotpie(df, f'Role distribution for {cohort}')
    img1 = addPlotImage(fig1, 0.6)
    elements.append(img1)
    elements.append(Spacer(1, 24))
    fig2 = plotages(df, f'Age distribution for {cohort}')
    img2 = addPlotImage(fig2, 0.6)
    elements.append(img2)
    fig3 = plotagegroups(df, f'{cohort}')
    img3 = addPlotImage(fig3, 0.85)
    elements.append(img3)
    elements.append(PageBreak())
    elements.extend(createGeneralReport(df, cohort, folder))
    elements.append(PageBreak())
    # create a table for each clinic
    for clinic in df['clinic'].unique():
        dfclinic = df[df['clinic'] == clinic]
        elements.append(createfronttable(dfclinic, f'{clinic} Summary', type_='clinic'))
        elements.append(Spacer(1, 24))
    doc.build(elements)
    # return elements

createcohortreport(dfboh3, 'BOH3')
createcohortreport(dfdds4, 'DDS4')

### Create student wise report

In [ ]:
def createEvalDf(row):
    evaldf = pd.DataFrame(columns = ['Key Competency', 'Marking Checklist', 'Student', 'Educator'])
    for stucol, educol in zip(studentcol, supervisorcol):
        # get common text in fullnamedict[stucol] and fullnamedict[educol]
        # commonText = 
        markingchecklist = fullnamedict[stucol].split(' Student Evaluation - ')[1]
        key = ''.join(markingchecklist.split('\n')[:-1])
        markingchecklist = markingchecklist.split('\n')[-1]
        print(key, markingchecklist)
        student = row[stucol]
        educator = row[educol]
        evaldf = pd.concat([evaldf, pd.DataFrame({'Key Competency':key, 'Marking Checklist': [markingchecklist], 
                                                  'Student': [student], 'Educator': [educator]})])
    return evaldf



def createReport(studentDf, id, name, savefolder):
    os.makedirs(savefolder, exist_ok=True)
    print(id, name)
    elements = []
    doc = SimpleDocTemplate(f'{savefolder}\\{name} ({id}).pdf', pagesize= pageSize, leftMargin = leftMargin,
                            rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    elements.append(Paragraph(f'{name} ({id})', headingStyle))
    elements.append(Spacer(1, 24))
    dfDateRoleItems = pd.DataFrame(columns=['Date', 'Role', 'Items', 'No. of Patients'])
    for i, row in studentDf.iterrows():
        date = row['date'].strftime('%d-%m-%Y')
        role = row['role']
        items = []
        for col in colCodeNew:
            items.extend(row[col])
        items = [item for item in items if item != 'nan']
        items = list(set(items))    
        # convert to string
        items = ', '.join(items).strip(',')
        # get the number of patients
        npts = getNPatients(row[colNPts.lower()])
        # get the clinical incident
        dfDateRoleItems = pd.concat([dfDateRoleItems, pd.DataFrame({'Date': [date], 'Role': [role], 'Items': [items], 'No. of Patients': [npts]})])
    display(dfDateRoleItems)
    table = createTable(dfDateRoleItems, 'Item Summary', colRatio = [1, 1.5, 3, 2], customTextCols=[2])
    elements.append(table)
    elements.append(PageBreak())
    studentDf.reset_index(drop=True, inplace=True)
    for i, row in studentDf.iterrows():
        # for all the important columns display column name and value
        elements.append(Paragraph(f'Form {i+1}: {row["date"].strftime("%d-%m-%Y")}', subheadingStyle))
        elements.append(Paragraph(f'Role: {row["role"]}', subsubheadingStyle))
        elements.append(Spacer(1, 10))
        # fill in patient info in colPtInfo
        lastPatient = None  # Keep track of the last patient number displayed

        for col in colPtInfo:
            if pd.isna(row[col.lower()]) or row[col.lower()] == 'nan':
                continue
            
            # Extract patient number using regex
            match = re.search(r'pt\.(\d+)', col.lower())
            if match:
                patientNum = match.group(1)
                
                # Check if this is a new patient number
                if lastPatient != patientNum:
                    elements.append(Paragraph(f'Patient {patientNum}', subsubheadingStyleL))  # Add bold subheading
                    elements.append(Spacer(1, 12))  # Add some space after the heading
                    lastPatient = patientNum  # Update the last patient tracker
            
            # Add patient info
            if 'code' in col.lower() or 'categories' in col.lower():
                continue
            elements.append(Paragraph(f'{fullnamedict[col]}: {row[col]}', tableTextStyleL))
            elements.append(Spacer(1, 6))
        # fill in the important columns
        for col in importantCols:
            if pd.isna(row[col.lower()]) or row[col.lower()] == 'nan':
                continue
            print(f'{col}: {row[col.lower()]}')
            elements.append(Paragraph(f'{col.strip()}', subsubheadingStyleL))
            elements.append(Spacer(1, 6))
            text = row[col.lower()].strip()
            # remove non-ascii characters
            text = re.sub(r'[^\x00-\x7F]+', '', text)
            elements.append(Paragraph(f'{text}', tableTextStyleL))
            elements.append(Spacer(1, 12))

        if row[colRole.lower()] == 'Operator':
            # elements.append(Paragraph(f'Evaluation', subheadingStyle))
            evaldf = createEvalDf(row)
            table = createTable(evaldf, 'Evaluation', colRatio = [2, 5, 1, 1], customTextCols=[0, 1], cellHighlight=True, 
                                tableTextStyle=tableTextStyleSmall)
            elements.append(table)


        elements.append(PageBreak())
        # break

    doc.build(elements)


    pass

# BOH3 first
def createStudentReport(df, cohort, savefolder):
    # print(dfboh3.columns)
    idCounts = df[colId].value_counts()
    ids = idCounts.index
    print(idCounts)
    for id in ids:
        studentDf = df[df[colId]==id]
        studentName = studentDf['family name'].values[0] + ' ' + studentDf['given name'].values[0]
        studentDf['date'] = pd.to_datetime(studentDf['date'])
        # display(studentDf)
        createReport(studentDf, id, studentName, os.path.join(savefolder, cohort))
        # break

savefolder = f'{folder}\\Student Reports'
createStudentReport(dfboh3, 'BOH3', savefolder) 
createStudentReport(dfdds4, 'DDS4', savefolder)
    

In [ ]:
# email the students their reports
import win32com.client
fullpath = 'C:/Users/Kunal Patel/D folder/MDS Work\\2025/BOH3+DDS4/Session 1/Student Reports'
folderboh3 = f'{fullpath}\\BOH3' # reports saved as name (id).pdf
folderdds4 = f'{fullpath}\\DDS4'
colemail = 'email'
colId = 'student id'
# Convert student IDs to string for safe comparison
dfboh3[colId] = df[colId].astype(str)
dfdds4[colId] = df[colId].astype(str)
# display(df[colId])
# display(df[df[colId] == '1473169'])
# Initialize Outlook
outlook = win32com.client.Dispatch("Outlook.Application")

# Iterate through folders
# for folder in foldero:
def sendEmails(folder, df, colId, colEmail):
    for file in os.listdir(folder):
        if file.endswith(".pdf"):
            # Extract student ID from filename (assumes format: Name (ID).pdf)
            match = re.search(r"\((\d+)\)", file)
            
            if not match:
                print(f"Skipping file (ID not found): {file}")
                continue
            print(f"Processing file: {file}========")
            studentId = match.group(1)  # Extracted ID
            studentId = str(studentId.strip())
            # print(studentId)
            # check studentId in df[colId]
            # for i, row in df.iterrows():
            #     if row[colId] == studentId:
            #         print(f"Student ID {studentId} found in the dataframe")
                    # break
            # Find email corresponding to student ID
            emailDf = df[df[colId] == studentId]
            # display(emailDf)    
            if emailDf.empty:
                print(f"No email found for Student ID: {studentId}")
                continue
            
            studentEmail = emailDf.iloc[0][colEmail]
            # studentEmail = 'kunal.patel0999@gmail.com'
            # Prepare email
            mail = outlook.CreateItem(0)
            mail.To = studentEmail
            mail.Subject = "DDS4/BOH3 Session 1 Report"
            mail.Body = f"Dear Student,\n\nPlease find attached your report.\n\nBest regards,\nKunal Patel"
            
            # Attach the report
            reportPath = os.path.join(folder, file)
            # print(reportPath)
            mail.Attachments.Add(reportPath)

            # Send email
            mail.Send()
            print(f"Email sent to {studentEmail} with attachment {file}")
            # break
    print("All emails processed!\n")

sendEmails(folderboh3, dfboh3, colId, colemail)
sendEmails(folderdds4, dfdds4, colId, colemail)

#### Clinic wise data

In [ ]:
def clinicReport(df, cohort):
    clinics = df['clinic'].unique()
    print(clinics)
    # display(df[df['clinic']=='Other'])
    ## replace Other with values in q58 column
    # df.loc[df['clinic']=='Other', 'clinic'] = df.loc[df['clinic']=='Other', 'q58']
    # print(df['clinic'].unique())
    # replace 'Primary Care - Emergency' 'PC emergency' 'PC Emergency' 'Primary care emergency' wiht 'PC Emergency'
    df.loc[df['clinic'].str.contains('Emergency', case=False), 'clinic'] = 'PC Emergency'
    reportdf = pd.DataFrame(columns = ['Clinic', '# Students', 'Items', 'Avg. # Patients', 'Avg. Entrustment', '# FTA'])
    tabledata = [reportdf.columns.to_list()[:2]]
    clinicgraphdict = {}
    for clinic in clinics:
        dfclinic = df[df['clinic']==clinic][df['role']=='Operator'][df['finished']==True]
        if len(dfclinic)==0:
            continue
        print(clinic)
        # if clinic!=' Northeast Health':
        #     continue
        # display(dfclinic)
        dfclinic['#patients'] = dfclinic[colNPts.lower()].apply(getNPatients)
        nfta = dfclinic[dfclinic['#patients']=='Patient FTA'].shape[0]
        dfclinic['#patients'] = dfclinic['#patients'].replace('Patient FTA', 0).astype(int)
        # drop 0 patients
        dfclinic = dfclinic[dfclinic['#patients']!=0]
      
        nstudents = len(dfclinic[dfclinic['role']=='Operator'][colId.lower()].unique())
        # get all the items
        items = []
        for col in colCodeNew:
            for item in dfclinic[col]:
                items.extend(item)
        items = [item for item in items if item != 'nan']

        items = [code for code in items if len(code) <= 5]
        # strip , . or space
        items = [item.strip(',. ') for item in items]

        # create a graph of items
        fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]*0.4))
        code_counts = pd.Series(items).value_counts(ascending=False)
        total = len(items)
        # remove counts less than 2

        sns.barplot(x=code_counts.index, y=code_counts.values, ax=ax)
        ax.set_title(None)
        ax.set_xlabel('Items')
        ax.set_ylabel('Frequency')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
        ax.text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        # y ticks should be integers
        ax.yaxis.get_major_locator().set_params(integer=True)
        plt.tight_layout()
        img = addPlotImage(fig, 0.9)
        if len(items)!=0:
            clinicgraphdict[clinic] = img
        items = list(set(items))
        items = ', '.join(items).strip(',')
        avgpatients = round(dfclinic[dfclinic['role'] == 'Operator']['#patients'].mean(), 2)
        dfclinic['entrustment'] = dfclinic['entrustment'].str.extract(r'Lvl (\d+)').astype(float)
        avgentrustment = dfclinic[dfclinic['role'] == 'Operator']['entrustment'].mean()
        avgentrustment = round(avgentrustment, 2)
        reportdf = pd.concat([reportdf, pd.DataFrame({'Clinic': [clinic], '# Students': [nstudents],
                                    'Items': [items], 'Avg. # Patients': [avgpatients], 'Avg. Entrustment': [avgentrustment],
                                    '# FTA': [nfta]})])
        tabledata.append([clinic, img])
        # break
    # display(reportdf)
    doc = SimpleDocTemplate(f'{folder}\\Clinic Summary {cohort}.pdf', pagesize= pageSize, leftMargin = leftMargin,
                                rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    elements = []
    # swith 3rd and 4th columns
    reportdf = reportdf.iloc[:, [0, 1, 3, 2] + list(range(4, reportdf.shape[1]))]
    table = createTable(reportdf, f'Clinic Summary {cohort}', colRatio = [1, 0.9, 1.3, 3.8, 1.5, 0.6], customTextCols=[0,3])
    # second table with tabledata
    colRatio = [1, 9]
    table2 = Table(tabledata, colWidths = [ratio/sum(colRatio) * variableUtils.pageSize[0] * 0.9 for ratio in colRatio])
    table2.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 0.2 * inch),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
        ('BOTTOMPADDING', (2, 1), (2, -1), 0),  
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    elements.append(PageBreak())
    # elements.append(table2)

    # add the clinic graphs
    elements.append(Paragraph('Item Distribution for Clinics', subheadingStyle))
    elements.append(Spacer(1, 18))  
    for clinic, img in clinicgraphdict.items():
        header = Paragraph(clinic, subsubheadingStyle)
        # elements.append(Paragraph(clinic, subheadingStyle))
        # elements.append(Spacer(1, 12))
        elements.append(KeepTogether([header, Spacer(1, 12), img]))
        # elements.append(PageBreak())
        elements.append(Spacer(1, 12))

    doc.build(elements)

clinicReport(dfboh3, 'BOH3')
clinicReport(dfdds4, 'DDS4')
clinicReport(df, 'BOH3+DDS4')

## DDS3

### GP and Perio

In [ ]:
workbookpath = '2025\DDS3\DDS3 CAF 2025_February 14, 2025_14.11.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df=pd.read_excel(workbookpath)
row0 = df.loc[0]
fullnamedict = {col: row0[col] for col in df.columns}
df.drop(0, inplace=True)
# for i, col in enumerate(df.columns):
#     print(i, col)
dfcolumns = df.columns.to_list()
# student details
studentinfoCols = dfcolumns[17:23]
studentIdCol = dfcolumns[19]
nameGCol = dfcolumns[17]
nameFCol = dfcolumns[18]
dateCol = dfcolumns[22]
# da details
roleCol = 'STUDENT ROLE'
roleClinicLoc = 'CLINIC LOC'
roleClinicType = 'CLINIC TYPE'
nopatientreason = 'DA NO PATIENT REASON'
nopatientreasonother = 'DA NO PATIENT REASON_4_TEXT'
# if other is selected in nopatientreason, then use nopatientreasonother
OTHER_TEXT = 'Other (please specify)'
df.loc[df[nopatientreason]==OTHER_TEXT, nopatientreason] = df.loc[df[nopatientreason]==OTHER_TEXT, nopatientreasonother]
# student evaluation cols
daEvalColsStu = dfcolumns[30:33]
daEvalColsSup = dfcolumns[33:36]
daICStu = dfcolumns[36]
daICExpStu = dfcolumns[37]
daReflecStu = dfcolumns[38]
# supervisor evaluation cols
daICSup = dfcolumns[39]
daICExpSup = dfcolumns[40]
daReflecSup = dfcolumns[43]
daNameSup = dfcolumns[44]
daGold = dfcolumns[42]    # student getting gold 
daIssues = dfcolumns[41]  # student having issues

# appicable clinics = GP Perio
clinics = ['GP', 'Perio']
# clinics = ['GP']
clinics = ['Perio']
# clinics=['Fixed Pros']
df = df[df[roleClinicType].isin(clinics)]
# df = df[df[roleClinicType]=='GP']
# df = df[df[roleClinicType]=='Perio']


In [ ]:
# Function to map reasons to categories
def categorize_reason(reason):
    if pd.isna(reason):
        return np.nan
    reason = reason.lower()
    if 'cancel' in reason or 'fta' in reason:
        return 'Cancellation'
    elif 'no ' in reason and 'book' in reason:
        return 'No Booking'
    elif 'fill' in reason:
        return 'Unable to Fill'
    elif 'assistant' in reason:
        return 'Assigned as Assistant'
    return 'Other'
def getGoldRatio(df, goldCol= daGold):
    info = {'number': df[goldCol].str.contains('Yes').sum()}
    return info

def getIssueRatio(df, issuesCol= daIssues):
    info = {'number': df[issuesCol].str.contains('Yes').sum()}
    return info

def getRoleCounts(df):
    return df[roleCol].value_counts()

In [ ]:
nTotal = len(df) # total forms
nStudents = len(df[studentIdCol].unique()) # number of students
dfidcounts = df[studentIdCol].value_counts().to_frame() # get student id values

# get role pie chart
fig, ax = plt.subplots(figsize=figSize)
df[roleCol].value_counts().plot.pie(ax=ax, autopct=lambda x: autopct(x, nTotal))
plt.title('Role Distribution')
plt.text(0.9, 0.9, f'Total: {nTotal}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
# plt.show()
imgrolepie = addPlotImage(fig, 0.5)

# get clinic pie chart
fig, ax = plt.subplots(figsize=figSize)
df[roleClinicType].value_counts().plot.pie(ax=ax, autopct=lambda x: autopct(x, nTotal))
plt.title('Clinic Type Distribution')
plt.text(0.9, 0.9, f'Total: {nTotal}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
# plt.show()
imgclinicpie = addPlotImage(fig, 0.5)

# create student, total, roles df===================================================================================
rolecountDf = pd.DataFrame(columns = ['Student ID', 'Student Name', 'Total Forms'] + df[roleCol].unique().tolist()[::-1])
# rolecountDf = pd.DataFrame(columns = ['Student ID', 'Student Name', 'Total Forms'] )
for id in dfidcounts.index:
    studentDf = df[df[studentIdCol]==id]
    studentName = studentDf[nameGCol].values[0] + ' ' + studentDf[nameFCol].values[0]
    totalforms = len(studentDf)
    rolecount = studentDf[roleCol].value_counts().astype(int)
    nopatient = studentDf[nopatientreason].value_counts().sum()
    rolecountDf = pd.concat([rolecountDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName],
                                                         'Total Forms': [totalforms], **rolecount.to_dict()})])
rolecountDf.fillna(0, inplace=True)
display(rolecountDf.head())
rolecounttable = createTable(rolecountDf, 'Role Counts', colRatio = [1, 3, 1] + [1]*len(df[roleCol].unique()), customTextCols=[1],
                             bottomPadding=8)

# get bar chart of no patient reasons===============================================================================================
print(df[nopatientreason].unique())
df['category'] = df[nopatientreason].apply(categorize_reason) # Apply the categorization function
nNoPatient = df['category'].value_counts()
# print(nNoPatient)
totalnopatient = nNoPatient.sum()
nNoPatientdf = nNoPatient.to_frame().reset_index()
nNoPatientdf.columns = ['Reason', 'Count']
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]*0.45))
sns.barplot(x='Count', y='Reason', data=nNoPatientdf, ax=ax, orient='h')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, ha='right')
ax.set_title('No Patient Reasons')
# show total number of patients
plt.text(0.9, 0.50, f'Total: {totalnopatient}/{nTotal}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
# show numbers on bars
for i, v in enumerate(nNoPatientdf['Count']):
    ax.text(v + 0.2, i, str(v), color='black')
fig.tight_layout()
plt.show()
imgnofill = addPlotImage(fig, 0.9)

In [ ]:
# evalution counts for each column
plt.ioff()

# get all unique values in oclumns daEvalColsStu
def getEvalCounts(df, evalCols):
    uniquevals = []
    for col in evalCols:
        uniquevals.extend(df[col].unique())
    uniquevals = list(set(uniquevals))
    # remove nan
    uniquevals = [val for val in uniquevals if not pd.isna(val)]
    # create df for evaluation counts for each column
    evalcountDf = pd.DataFrame(columns = ['Question'] + uniquevals)
    for col in evalCols:
        evalcount = df[col].value_counts()
        fulltext = fullnamedict[col].split('-')[-1].replace('\n\n', '\n')
        evalcountDf = pd.concat([evalcountDf, pd.DataFrame({'Question': [fulltext],
                                                             **evalcount.to_dict()})])
    evalcountDf.fillna(0, inplace=True)
    return evalcountDf

evalcountDfStu = getEvalCounts(df, daEvalColsStu)
evalcountDfSup = getEvalCounts(df, daEvalColsSup)
# replace Yes withih guidance with Yes with guidance
evalcountDfSup.rename(columns={'Yes witih guidance': 'Yes with guidance'}, inplace=True)
display(evalcountDfStu)
display(evalcountDfSup)
colorDict = {
    'Yes': '#4c72b0',             # Muted Blue
    'Yes with guidance': '#55a868',  # Soft Green
    'No': '#e07a5f',             # Warm Red-Orange
    'Not assessed': '#bfbfbf',    # Neutral Grey
    'NA': '#bfbfbf'              # Grey for renamed 'Not assessed'
}

# crate pie charts for both student and supervisor with each having subplots for counts for each question
def createEvalPieChart(df, title):
    fig, ax = plt.subplots(
        nrows=df.shape[0], 
        ncols=1, 
        figsize=(figSize[0]*0.5, figSize[1]*0.8), 
        sharex=True
    )
    
    df.reset_index(drop=True, inplace=True)
    df.rename(columns={"Not assessed": "NA"}, inplace=True)
    # remove columns with 0 counts
    for i, row in df.iterrows():
        # remove columns with 0 counts
        row = row[row != 0]
        labels = row[1:].index
        sizes = row[1:].values
        # strip the labels
        labels = [label.strip() for label in labels]
        print(labels)
        # Get colors from colorDict or default
        pieColors = [colorDict[label] for label in labels]
        
        ax[i].pie(
            sizes, 
            autopct=lambda x: autopct(x, sizes.sum()), 
            labels=labels, 
            colors=pieColors, 
            textprops={'fontsize': 8}
        )
        ax[i].set_title(row['Question'], fontsize=7)
    
    # Increase the space between subplots
    plt.subplots_adjust(hspace=0.4)
    plt.suptitle(title, fontsize=16, y=1.02)
    
    return fig


figstu = createEvalPieChart(evalcountDfStu, 'Student Evaluation Counts')
figsup = createEvalPieChart(evalcountDfSup, 'Supervisor Evaluation Counts')

def figToImage(fig):
    """Converts a Matplotlib figure to a PIL Image"""
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    return PIL.Image.open(buf)

# Convert both figures to images
imgStu = addPlotImage(figstu, 0.9)
imgSup = addPlotImage(figsup, 0.9)

# # Create a new figure with 1 row and 2 columns
# fig, axes = plt.subplots(1, 2, figsize=(figSize[0], figSize[1] * 0.8))

# # Display images in the subplots
# axes[0].imshow(imgStu)
# axes[0].axis('off')
# # axes[0].set_title('Student Evaluation Counts')

# axes[1].imshow(imgSup)
# axes[1].axis('off')
# # axes[1].set_title('Supervisor Evaluation Counts')

# plt.tight_layout()
# evalImg = addPlotImage(fig, 0.9)

# get gold and issue ratios
goldratio = getGoldRatio(df)
issueratio = getIssueRatio(df)
nAssistant = df[df[roleCol]=='Assistant'].shape[0]
print(goldratio, issueratio)

# IC breaches count
icbreach = df[daICSup].str.contains('Yes').sum()

# student feedback
# find non na values in daStuReflec
nNonNaStu = df[daReflecStu].notna().sum()
nNonNaSup = df[daReflecSup].notna().sum()
# find in role Assistant and Operator
nNonNaStuOper = df[df[roleCol]=='Operator'][daReflecStu].notna().sum()
nNonNaStuAssist = df[df[roleCol]=='Assistant'][daReflecStu].notna().sum()
# get number of positive and negative feedback (to do)

# get supervisor name value counts
supervisors = df[daNameSup].value_counts()
print(supervisors)
# Supervisor info is not valid as most only checked one student each


#### OP info now

In [ ]:
serviceCols = dfcolumns[60:68]
colServiceGeneral = dfcolumns[68]
additionalCols = dfcolumns[69:79]
complexityColStu = dfcolumns[81]
complexityColSup = dfcolumns[82]
colGeneralEvalStu = dfcolumns[84:92]
colGeneralEvalSup = dfcolumns[92:100]
colClnincEvalStu = dfcolumns[100:155]
colClnincEvalSup = dfcolumns[155:210]
opICStu = dfcolumns[210]
opICExpStu = dfcolumns[211]
opReflecStu = dfcolumns[212]
opICSup = dfcolumns[213]
opICExpSup = dfcolumns[214]
opOverallComp = dfcolumns[215]
opIssues = dfcolumns[216]
opGold = dfcolumns[217]
opReflecSup = dfcolumns[218]
opNameSup = dfcolumns[219]

In [ ]:

# Item information
def process_row(row, serviceCols, colServiceGeneral, additionalCols):
    allCodes = []
    # Extract codes from the service columns
    for col in serviceCols:
        if col in row:
            allCodes += extractCodes(row[col], 'Clinic')
    
    # Extract general service codes and modifiers
    if colServiceGeneral in row:
        general_codes = extractGeneralServiceCode(row[colServiceGeneral])
        general_codes = [(code, location.upper()) for code, location in general_codes]
        allCodes += general_codes
    
    # Format the final string
    
    for col in additionalCols:
        if row[col] not in [np.nan, 'nan']:
            allCodes.append((str(row[col]), 'clinic'))  # Add additional codes
    # if there are 2 digit codes, add a 0 in front
    allCodes = [(f'0{code}', location) if len(code) == 2 else (code, location) for code, location in allCodes]
    codeString = ', '.join([f"{code}" for code, location in allCodes])
    return codeString


laCol = 'LA'
df['ItemsOp'] = df.apply(lambda row: process_row(row, serviceCols, colServiceGeneral, additionalCols), axis=1)
# add LA to string if Yes
df['ItemsOp'] = df.apply(lambda row: f'{row["ItemsOp"]}, LA' if row[laCol]=='Yes' else row['ItemsOp'], axis=1)
# display(df['ItemsOp'].head(25))
# get bar chart of item counts
items = []
for item in df['ItemsOp']:
    items.extend(item.split(', '))
items = [item.strip() for item in items]
# remove nan
items = [item for item in items if item != 'nan']
# remove if more than 5 characters
items = [item for item in items if len(item) <= 5]
# create a graph of items
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]*0.6))
code_counts = pd.Series(items).value_counts(ascending=False)
# remove empty strings
code_counts = code_counts[code_counts.index != '']
total = len(items)
# plot
sns.barplot(x=code_counts.index, y=code_counts.values, ax=ax)
ax.set_title('Item Distribution')
ax.set_xlabel('Items')
ax.set_ylabel('Frequency')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
ax.text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
# y ticks should be integers
ax.yaxis.get_major_locator().set_params(integer=True)
plt.tight_layout()
imgItemDist = addPlotImage(fig, 0.9)

# complexity count pie charts for student and supervisors in 2 subplots
complexityCountsStu = df[complexityColStu].value_counts()
complexityCountsSup = df[complexityColSup].value_counts()
totalStu = complexityCountsStu.sum()
totalSup = complexityCountsSup.sum()
fig, axes = plt.subplots(1, 2, figsize=(figSize[0], figSize[1]*1.5))
axes[0].pie(complexityCountsStu, autopct=lambda x: autopct(x, totalStu), labels=complexityCountsStu.index)
axes[0].set_title('Student Complexity')
axes[1].pie(complexityCountsSup, autopct=lambda x: autopct(x, totalSup), labels=complexityCountsSup.index)
axes[1].set_title('Supervisor Complexity')
# write the total number of values
axes[0].text(0.9, 0.9, f'Total: {totalStu}', horizontalalignment='center', verticalalignment='center', transform=axes[0].transAxes)
axes[1].text(0.9, 0.9, f'Total: {totalSup}', horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes)
plt.tight_layout()
imgComplexity = addPlotImage(fig, 0.9)

# if both student and supervisor have the different complexity, bar the nan values add a flag to complexityflag column
df['complexityflag'] = df[complexityColStu].fillna('nan') != df[complexityColSup].fillna('nan')
# get the counts
complexityflag = df['complexityflag'].value_counts()
print(complexityflag)
# display(df[df['complexityflag']].head())

# get general evaluation counts for student and supervisor columns and plot a stacked bar chart
uniquevals = [ 'Yes', 'Yes with guidance','No', 'Not assessed']
colors = ['blue', 'green', 'orange', 'lightgrey']
colors = ['#4c72b0',  # Muted Blue for 'Yes'
          '#55a868',  # Soft Green for 'Yes with guidance'
          '#e07a5f',  # Warm Red-Orange for 'No'
          '#bfbfbf']  # Neutral Grey for 'Not assessed'

def getEvalCounts(df, colGeneralEval):
    # create df for evaluation counts for each column
    evalcountDf = pd.DataFrame(columns = ['Question'] + uniquevals)
    for col in colGeneralEval:
        evalcount = df[col].value_counts()
        fulltext = fullnamedict[col].split('-')[-1].replace('\n\n', '\n')
        evalcountDf = pd.concat([evalcountDf, pd.DataFrame({'Question': [fulltext],
                                                             **evalcount.to_dict()})])
    evalcountDf.fillna(0, inplace=True)
    # sort by the total sum of the columns
    evalcountDf['sum'] = evalcountDf[uniquevals].sum(axis=1)
    evalcountDf = evalcountDf.sort_values('sum', ascending=False).drop('sum', axis=1)
    evalcountDf = evalcountDf.head(15)
    return evalcountDf


# create stacked bar chart for general evaluation counts
def createStackedBarChart(df, title, plotCol):
    fig, ax = plt.subplots(figsize=(figSize[0]*0.6, figSize[1]*0.8))
    plotdf = df.set_index(plotCol)
    plotdf = plotdf.loc[::-1]  # Reverse the order of bars
    plotdf.plot(kind='barh', stacked=True, ax=ax, color=colors)
    ax.set_title(title)
    ax.set_xlabel('Questions')
    ax.set_ylabel('Counts')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
    ax.legend(loc='center left', bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()
    return fig

# Create pseudo labels for questions

def createPseudoLabels(df):
    pseudo_labels = {}
    for i, question in enumerate(df['Question']):
        pseudo_label = f'MC{i+1}'
        pseudo_labels[question] = pseudo_label
    return pseudo_labels

# Apply pseudo labels to the dataframe
def applyPseudoLabels(df, pseudo_labels):
    df['PseudoLabel'] = df['Question'].map(pseudo_labels)
    return df

# Create reference table
def createReferenceTable(pseudo_labels):
    reference_table = pd.DataFrame(list(pseudo_labels.items()), columns=['Original Question', 'Pseudo Label'])
    return reference_table

# Generate pseudo labels and apply them
def createEvalPlots(df, colGeneralEvalStu, colGeneralEvalSup):
    evalcountDfStu = getEvalCounts(df, colGeneralEvalStu)
    evalcountDfSup = getEvalCounts(df, colGeneralEvalSup)
    display(evalcountDfStu.head())
    display(evalcountDfSup.head())
    pseudo_labels_stu = createPseudoLabels(evalcountDfStu)
    pseudo_labels_sup = createPseudoLabels(evalcountDfSup)
    evalcountDfStu = applyPseudoLabels(evalcountDfStu, pseudo_labels_stu)
    evalcountDfSup = applyPseudoLabels(evalcountDfSup, pseudo_labels_sup)

    # Create reference tables
    reference_table_stu = createReferenceTable(pseudo_labels_stu)
    reference_table_sup = createReferenceTable(pseudo_labels_sup)

    # Display reference tables
    # display(reference_table_stu)
    # display(reference_table_sup)

    # Create stacked bar charts with pseudo labels
    figstu = createStackedBarChart(evalcountDfStu[['PseudoLabel'] + uniquevals], 'Student General Evaluation', 'PseudoLabel')
    figsup = createStackedBarChart(evalcountDfSup[['PseudoLabel'] + uniquevals], 'Supervisor General Evaluation', 'PseudoLabel')

    figstu = createStackedBarChart(evalcountDfStu, 'Student General Evaluation Counts', 'PseudoLabel')
    figsup = createStackedBarChart(evalcountDfSup, 'Supervisor General Evaluation Counts', 'PseudoLabel')
    # Convert both figures to images
    imgStu = figToImage(figstu)
    imgSup = figToImage(figsup)
    # Create a new figure with 1 row and 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(figSize[0], figSize[1]))
    # Display images in the subplots
    axes[0].imshow(imgStu)
    axes[0].axis('off')
    axes[1].imshow(imgSup)
    axes[1].axis('off')
    plt.tight_layout()
    evalImgOp = addPlotImage(fig, 0.9)
    refTable = createTable(reference_table_stu, 'Reference Table', colRatio = [4, 1], customTextCols=[0])
    return evalImgOp, refTable

evalImgOp, refTable = createEvalPlots(df, colGeneralEvalStu, colGeneralEvalSup)
evalImgOp2, refTable2 = createEvalPlots(df, colClnincEvalStu, colClnincEvalSup)

# get IC breaches count
icbreachdf = df[df[opICSup]=='Yes']
icbreachdf.rename({studentIdCol: 'Student ID'}, axis=1, inplace=True)
icbreachdf['Student Name'] = icbreachdf[nameGCol] + ' ' + icbreachdf[nameFCol]
icbreachdf = icbreachdf[['Student ID', 'Student Name', opICExpSup]]
# icbreachids = icbreachdf[studentIdCol].unique()
# find ids and names of students with IC breaches
# print(icbreachids)
# icbreachnames = icbreachdf[nameGCol] + ' ' + icbreachdf[nameFCol]
# icbreachdf = pd.DataFrame({'Student ID': icbreachids, 'Student Name': icbreachnames})
icbreachtable = createTable(icbreachdf, 'Students with IC Breaches', colRatio = [1, 2, 4], customTextCols=[1, 2])
# get gold and issue ratios
opgoldratio = getGoldRatio(df, goldCol=opGold)
opIssuesratio = getIssueRatio(df, issuesCol=opIssues)
nOperator = df[df[roleCol]=='Operator'].shape[0]
# get overall competency counts and plot a pie chart
opOverallCompCounts = df[opOverallComp].value_counts()
totalOp = opOverallCompCounts.sum()
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]*0.8))
ax.pie(opOverallCompCounts, autopct=lambda x: autopct(x, totalOp), labels=opOverallCompCounts.index)
ax.set_title('Overall Competency')
plt.tight_layout()
imgOverallComp = addPlotImage(fig, 0.9)



In [ ]:
# analyse info per row/student
# get studentid value counts
studentidcounts = df[studentIdCol].value_counts()
print(studentidcounts)
# for each row get Yes No and Not assessed counts for geneeral evaluation and clinic evaluation
# first for general evaluation
df['Yes General Stu'] = df[colGeneralEvalStu].apply(lambda x: (x == 'Yes').sum(), axis=1)
df['Yes with guidance General Stu'] = df[colGeneralEvalStu].apply(lambda x: (x == 'Yes with guidance').sum(), axis=1)
df['No General Stu'] = df[colGeneralEvalStu].apply(lambda x: (x == 'No').sum(), axis=1)
df['Not assessed General Stu'] = df[colGeneralEvalStu].apply(lambda x: (x == 'Not assessed').sum(), axis=1)
# score is 1 for Yes, 1 for Yes with guidance, 0 for No and 0.5 for Not assessed (Yes + Yes with guidance)/(Yes + Yes with guidance + No + 0.5*Not assessed)
df['General Stu Score'] = (df['Yes General Stu'] + df['Yes with guidance General Stu'] + 0.5*df['Not assessed General Stu'])/(df['Yes General Stu'] + df['Yes with guidance General Stu'] + df['No General Stu'] + 0.5*df['Not assessed General Stu'])
# then for clinic evaluation
df['Yes Clinic Stu'] = df[colClnincEvalStu].apply(lambda x: (x == 'Yes').sum(), axis=1)
df['Yes with guidance Clinic Stu'] = df[colClnincEvalStu].apply(lambda x: (x == 'Yes with guidance').sum(), axis=1)
df['No Clinic Stu'] = df[colClnincEvalStu].apply(lambda x: (x == 'No').sum(), axis=1)
df['Not assessed Clinic Stu'] = df[colClnincEvalStu].apply(lambda x: (x == 'Not assessed').sum(), axis=1)
df['Clinic Stu Score'] = (df['Yes Clinic Stu'] + df['Yes with guidance Clinic Stu'] + 0.5*df['Not assessed Clinic Stu'])/(df['Yes Clinic Stu'] + df['Yes with guidance Clinic Stu'] + df['No Clinic Stu'] + 0.5*df['Not assessed Clinic Stu'])

# Plot the scores v/s overall competencies as two scatter plots one for student and one for supervisor
fig, ax = plt.subplots(1, 2, figsize=(figSize[0], figSize[1] * 0.45))

# Only include operators
dfscatter = df[df[roleCol] == 'Operator']
xCategories = dfscatter[opOverallComp].unique()
# General Evaluation Scatter + Box Plot
sns.boxplot(y='General Stu Score', x=opOverallComp, data=dfscatter, ax=ax[0], showcaps=False, 
            boxprops={'facecolor':'none'}, whiskerprops={'linewidth':0}, medianprops={'color':'black'})
sns.scatterplot(y='General Stu Score', x=opOverallComp, data=dfscatter, ax=ax[0], alpha=0.7)
ax[0].set_title('General Evaluation Score vs Overall Competency')

# Clinic Evaluation Scatter + Box Plot
sns.boxplot(y='Clinic Stu Score', x=opOverallComp, data=dfscatter, ax=ax[1], showcaps=False, 
            boxprops={'facecolor':'none'}, whiskerprops={'linewidth':0}, medianprops={'color':'black'})
sns.scatterplot(y='Clinic Stu Score', x=opOverallComp, data=dfscatter, ax=ax[1], alpha=0.7)
ax[1].set_title('Clinic Evaluation Score vs Overall Competency')

# Rotate x labels
for i in range(2):
    ax[i].set_xticklabels([label.get_text().split('/')[0] for label in ax[i].get_xticklabels()], rotation=90)

plt.tight_layout()
imgScatter1 = addPlotImage(fig, 0.9)

# now same for supervisor
# first for general evaluation
df['Yes General Sup'] = df[colGeneralEvalSup].apply(lambda x: (x == 'Yes').sum(), axis=1)
df['Yes with guidance General Sup'] = df[colGeneralEvalSup].apply(lambda x: (x == 'Yes with guidance').sum(), axis=1)
df['No General Sup'] = df[colGeneralEvalSup].apply(lambda x: (x == 'No').sum(), axis=1)
df['Not assessed General Sup'] = df[colGeneralEvalSup].apply(lambda x: (x == 'Not assessed').sum(), axis=1)
df['General Sup Score'] = (df['Yes General Sup'] + df['Yes with guidance General Sup'] + 0.5*df['Not assessed General Sup'])/(df['Yes General Sup'] + df['Yes with guidance General Sup'] + df['No General Sup'] + 0.5*df['Not assessed General Sup'])
# then for clinic evaluation
df['Yes Clinic Sup'] = df[colClnincEvalSup].apply(lambda x: (x == 'Yes').sum(), axis=1)
df['Yes with guidance Clinic Sup'] = df[colClnincEvalSup].apply(lambda x: (x == 'Yes with guidance').sum(), axis=1)
df['No Clinic Sup'] = df[colClnincEvalSup].apply(lambda x: (x == 'No').sum(), axis=1)
df['Not assessed Clinic Sup'] = df[colClnincEvalSup].apply(lambda x: (x == 'Not assessed').sum(), axis=1)
df['Clinic Sup Score'] = (df['Yes Clinic Sup'] + df['Yes with guidance Clinic Sup'] + 0.5*df['Not assessed Clinic Sup'])/(df['Yes Clinic Sup'] + df['Yes with guidance Clinic Sup'] + df['No Clinic Sup'] + 0.5*df['Not assessed Clinic Sup'])

# Plot the scores v/s overall competencies as two scatter plots one for student and one for supervisor
fig, ax = plt.subplots(1, 2, figsize=(figSize[0], figSize[1] * 0.45))

# Only include operators
dfscatter = df[df[roleCol] == 'Operator']
xCategories = dfscatter[opOverallComp].unique()
# General Evaluation Scatter + Box Plot
sns.boxplot(y='General Sup Score', x=opOverallComp, data=dfscatter, ax=ax[0], showcaps=False, 
            boxprops={'facecolor':'none'}, whiskerprops={'linewidth':0}, medianprops={'color':'black'})
sns.scatterplot(y='General Sup Score', x=opOverallComp, data=dfscatter, ax=ax[0], alpha=0.7)
ax[0].set_title('General Evaluation Score vs Overall Competency')

# Clinic Evaluation Scatter + Box Plot
sns.boxplot(y='Clinic Sup Score', x=opOverallComp, data=dfscatter, ax=ax[1], showcaps=False, 
            boxprops={'facecolor':'none'}, whiskerprops={'linewidth':0}, medianprops={'color':'black'})
sns.scatterplot(y='Clinic Sup Score', x=opOverallComp, data=dfscatter, ax=ax[1], alpha=0.7)
ax[1].set_title('Clinic Evaluation Score vs Overall Competency')

# Rotate x labels
for i in range(2):
    ax[i].set_xticklabels([label.get_text().split('/')[0] for label in ax[i].get_xticklabels()], rotation=90)
# don't show x axis title for both
ax[0].set_xlabel('')
ax[1].set_xlabel('')
plt.tight_layout()
imgScatter2 = addPlotImage(fig, 0.9)

# Unfinished students
colFinished = 'Finished'
Unfinished = df[df[colFinished]=='FALSE']
print(Unfinished)
Unfinishednames = Unfinished[nameGCol] + ' ' + Unfinished[nameFCol]
UnfinishedText = f'These student forms are unfinished: {", ".join(Unfinishednames)}'
# now save the df
# df.to_excel(f'{folder}/{file} new.xlsx', index=False)

In [ ]:
doc = SimpleDocTemplate(f'{folder}\\DDS3 {"+".join(clinics)} Summary.pdf', pagesize= pageSize, leftMargin = leftMargin,
                            rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
elements = []
elements.append(Paragraph('Summary', headingStyle))
elements.append(Spacer(1, 24))
elements.append(Paragraph('General Information', subheadingStyle))
elements.append(Spacer(1, 12))
elements.append(Paragraph(f'Clinics: {", ".join(clinics)}', subsubheadingStyle))
elements.append(Spacer(1, 24))
elements.append(Paragraph('Clinic Distribution', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(imgclinicpie)
elements.append(Spacer(1, 24))
elements.append(Paragraph('Role Distribution', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(imgrolepie)
elements.append(Spacer(1, 24))
elements.append(rolecounttable)
elements.append(KeepTogether([
    Paragraph('No Patient Reasons', subsubheadingStyle),
    Spacer(1, 12),
    imgnofill,
]))

elements.append(PageBreak())
# elements.append(Spacer(1, 24))
elements.append(Paragraph('DA Information', subheadingStyle))
elements.append(Spacer(1, 12))
elements.append(Paragraph('Evaluation Summary', subsubheadingStyle))
elements.append(Spacer(1, 24))
elements.append(imgStu)
elements.append(Spacer(1, 12))
elements.append(imgSup)
elements.append(PageBreak())
elements.append(Spacer(1, 24))
elements.append(Paragraph('Gold Stars and Issues', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(Paragraph(f'Number of Gold Stars: {goldratio["number"]}/{nAssistant}.', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Percentage of Gold Stars: {round(goldratio["number"]/nAssistant*100,2)}%', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Number of Issues Raised: {issueratio["number"]}/{nAssistant}', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Number of IC Breaches: {icbreach}/{nAssistant}', tableTextStyle))
elements.append(Spacer(1, 24))
elements.append(Paragraph(f'Student and Supervisor Reflections', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(Paragraph(f'Number of Student Reflections: {nNonNaStu}/{nAssistant}', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Number of Supervisor Reflections: {nNonNaSup}/{nAssistant}', tableTextStyle))
elements.append(Spacer(1, 12))
# elements.append(Paragraph(f'Supervisor grading Information', subsubheadingStyle))
# elements.append(Spacer(1, 6))
# elements.append(Paragraph(f'Supervisor Information is not valid as most only checked one student each', tableTextStyle))
elements.append(PageBreak())

# operator info now
elements.append(Paragraph('Operator Information', subheadingStyle))
elements.append(Spacer(1, 12))
elements.append(imgItemDist)
elements.append(Spacer(1, 12))
elements.append(Paragraph('Complexity Information', subsubheadingStyle))
elements.append(Spacer(1, 24))
elements.append(imgComplexity)
elements.append(Spacer(1, 12))
elements.append(Paragraph("Some supervisors are not filling in the complexity, IDK if this is a problem. For them I am using student's complexity level", tableTextStyle))
elements.append(Spacer(1, 12))

elements.append(KeepTogether([
    Paragraph('Evaluation Information General', subsubheadingStyle),
    Spacer(1, 12),
    evalImgOp,
    Spacer(1, 12),
    refTable,
    PageBreak()
]))

elements.append(KeepTogether([
    Paragraph('Evaluation Information Clinic', subsubheadingStyle),
    Spacer(1, 12),
    evalImgOp2,
    Spacer(1, 12),
    refTable2,
    PageBreak()
]))

elements.append(Paragraph('Overall Competency', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(imgOverallComp)
elements.append(Spacer(1, 24))

elements.append(Paragraph('Gold Stars and Issues', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(Paragraph(f'Number of Gold Stars: {opgoldratio["number"]}/{nOperator}.', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Percentage of Gold Stars: {round(opgoldratio["number"]/nOperator*100,2)}%', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Number of Issues Raised: {opIssuesratio["number"]}/{nOperator}', tableTextStyle))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f'Number of IC Breaches: {icbreach}/{nOperator}', tableTextStyle))
elements.append(Spacer(1, 24))
elements.append(icbreachtable)
elements.append(PageBreak())
elements.append(Paragraph('Scatter Plots for score v/s competency', subheadingStyle))
elements.append(Spacer(1, 24))
elements.append(Paragraph('Student Evaluation', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(imgScatter1)
elements.append(Spacer(1, 24))
elements.append(Paragraph('Supervisor Evaluation', subsubheadingStyle))
elements.append(Spacer(1, 12))
elements.append(imgScatter2)
elements.append(Spacer(1, 24))
elements.append(Paragraph('Correlation between overall competency and scores have improved but still the <i>Developing Competency</i> is too spread out in scores. Implies objectivity in marking has increased but still needs improvement.', tableTextStyle))
elements.append(PageBreak())
elements.append(Paragraph('Unfinished Students', subheadingStyle))
elements.append(Spacer(1, 12))
elements.append(Paragraph(UnfinishedText, tableTextStyle))

doc.build(elements)
import sys
sys.exit("Stopping execution")

### FP etc.

In [ ]:
workbookpath = '2025\DDS3\\2025 DDS3 Clinical Assessment Form_January 30, 2025_02.50.xlsx'
workbookpath = '2025\DDS3\Specialist\\2025 DDS3 Clinical Assessment Form (Specialist Clinics)_February 14, 2025_15.10.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath, keep_default_na=False, na_values=[''])  # This will keep 'NA' text as it is
# turn evey column to lowercase
df.columns = df.columns.str.lower()
# strip all columns
df.columns = df.columns.str.strip()
# remove non-ascii characters
df.columns = df.columns.str.replace(r'[^\x00-\x7F]+', '', regex=True)
df.loc[0] = df.loc[0].str.replace(r'[^\x00-\x7F]+', '', regex=True)
supervisorcol = [col for col in df.columns if 'evaluation#2' in col.lower()]
studentcol = [col for col in df.columns if 'evaluation#1' in col.lower()]
#other student col match Q\d+#1
otherstudentcol =([col for col in df.columns if re.match(r'q\d+#1', col)])
#other supervisor col match Q\d+#2
othersupervisorcol = ([col for col in df.columns if re.match(r'q\d+#2', col)])
allstudentcol = studentcol + otherstudentcol
allsupervisorcol = supervisorcol + othersupervisorcol

display(df[supervisorcol[3]].value_counts(dropna=False))
fullnamedict = {col: df.loc[0, col] for col in df.columns}
row0 = df.loc[0]
# row0 = row0.str.replace(r'[^\x00-\x7F]+', '', regex=True)
df.drop(0, inplace=True)
colAge = [col for col in df.columns if '#2_1_1' in col]
colPtInfo = [col for col in df.columns if re.match(r'pt\.\s*\d+', col)]

def getAgeGroup(age):
    if age <= 6:
        return '0-6'
    elif age <= 17:
        return '7-17'
    elif age >= 18 and age<140:
        return '18+'
    else:
        return np.nan
    
for col in colAge:
    df[col] = df[col].astype(float)
colAgeNew = []
for col in colAge:
    df[f'{col} group'] = df[col].apply(getAgeGroup)
    colAgeNew.append(f'{col} group')
# display(df[colAge+ colAgeNew])
colCode = [col for col in df.columns if 'code' in col.lower()]
print(colCode)
colCodeNew = []
for col in colCode:
    df[col] = df[col].astype(str)
    df[f'{col} list'] = df[col].apply(lambda x: [f'0{item}' if item.isdigit() and len(item) == 2 else item for item in re.split(r',\s*', x)])  # Handles optional spaces after commas and adds 0 before two-digit items
    colCodeNew.append(f'{col} list')
# display(df[colCodeNew])
# combine all the codes into one column, data is in the form of a list
df['Items'] = df.apply(lambda row: [item for sublist in row[colCodeNew].values for item in sublist], axis=1)
df['Items'] = df['Items'].apply(lambda x: [item for item in x if item != 'nan'])
# display(df[colCodeNew + ['Items']])
colCodeNew = ['Items']
colNPts = 'No. of pts'
colId = 'student id'
colOprStuFeedback = 'student reflection'
colCI = 'Clinical Incident'
colCIExp = 'CI Explanation'
# patient info columns are with Pt.\d+ in the column name
colDate = 'date'
df[colDate] = pd.to_datetime(df[colDate])

print(colPtInfo)
importantCols =  ['SO Feedback',	'SO Edu Feedback',	'SO Edu Name', 'Obs Feedback',	'Obs Edu Feedback', 'Obs Edu Name']
columnList = [
    "Student Reflection",
    "Edu Feedback",
    "Entrustment",
    "Clinical Incident",
    "CI Explanation",
    "Edu Name"
]
colRotation = 'clinic'
df.rename(columns={'rotation': colRotation}, inplace=True)
colrole = 'role'
importantCols.extend(columnList)
print(fullnamedict)
colNPtsNumber = 'no. of pts number'
def getNPatients(text):
    # print(text)
    if pd.isna(text):
        return 0
    # select the number of patients from the text, select the highest number Patient 1, Patient 2, Patient 3 etc.
    numbers = re.findall(r'\d+', text)
    if len(numbers) == 0 and 'fta' in text.lower():
        return 'Patient FTA'
    # get the highest number
    if len(numbers):
        return max([int(num) for num in numbers])
    else:
        return 0
colPriority = [col for col in df.columns if 'priority' in col.lower()]
colGivenName = 'given name'
colFamilyName = 'family name'
df[colNPtsNumber] = df[colNPts.lower()].apply(getNPatients)



In [ ]:
uniquevals = [ 'Yes','No', 'NA']
# colors = {'Yes': 'blue', 'No': 'orange', 'Not assessed': 'lightgrey', 'Not relevant': 'green'}
colors = ['blue', 'orange', 'lightgrey', 'green']
def figToImage(fig):
    """Converts a Matplotlib figure to a PIL Image"""
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    return PIL.Image.open(buf)

def getEvalCounts(df, colGeneralEval):
    # create df for evaluation counts for each column
    evalcountDf = pd.DataFrame(columns = ['Question'] + uniquevals)
    for col in colGeneralEval:
        evalcount = df[col].value_counts()
        fulltext = fullnamedict[col].split(' - ')[-1].replace('\n\n', '\n')
        evalcountDf = pd.concat([evalcountDf, pd.DataFrame({'Question': [fulltext],
                                                             **evalcount.to_dict()})])
    evalcountDf.fillna(0, inplace=True)
    # sort by the total sum of the columns
    evalcountDf['sum'] = evalcountDf[uniquevals].sum(axis=1)
    evalcountDf = evalcountDf.sort_values('sum', ascending=False)
    # remove rows with sum = 0
    evalcountDf = evalcountDf[evalcountDf['sum'] != 0]
    evalcountDf.drop('sum', axis=1, inplace=True)
    evalcountDf = evalcountDf.head(15)
    return evalcountDf


# create stacked bar chart for general evaluation counts
def createStackedBarChart(df, title, plotCol):
    fig, ax = plt.subplots(figsize=(figSize[0]*0.6, figSize[1]*0.8))
    plotdf = df.set_index(plotCol)
    plotdf = plotdf.loc[::-1]  # Reverse the order of bars
    # remove rows with all zeros
    plotdf = plotdf.loc[(plotdf!=0).any(axis=1)]
    plotdf.plot(kind='barh', stacked=True, ax=ax, color=colors)
    ax.set_title(title)
    ax.set_xlabel('Questions')
    ax.set_ylabel('Counts')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=8)
    ax.legend(loc='center left', bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()
    return fig

# Create pseudo labels for questions

def createPseudoLabels(df):
    pseudo_labels = {}
    for i, question in enumerate(df['Question']):
        pseudo_label = f'MC{i+1}'
        pseudo_labels[question] = pseudo_label
    return pseudo_labels

# Apply pseudo labels to the dataframe
def applyPseudoLabels(df, pseudo_labels):
    df['PseudoLabel'] = df['Question'].map(pseudo_labels)
    return df

# Create reference table
def createReferenceTable(pseudo_labels):
    reference_table = pd.DataFrame(list(pseudo_labels.items()), columns=['Original Question', 'Pseudo Label'])
    return reference_table

# Generate pseudo labels and apply them
def createEvalPlots(df, colGeneralEvalStu, colGeneralEvalSup):
    evalcountDfStu = getEvalCounts(df, colGeneralEvalStu)
    evalcountDfSup = getEvalCounts(df, colGeneralEvalSup)

    display(evalcountDfStu)
    # display(evalcountDfSup.head())
    pseudo_labels_stu = createPseudoLabels(evalcountDfStu)
    pseudo_labels_sup = createPseudoLabels(evalcountDfSup)
    evalcountDfStu = applyPseudoLabels(evalcountDfStu, pseudo_labels_stu)
    evalcountDfSup = applyPseudoLabels(evalcountDfSup, pseudo_labels_sup)

    # Create reference tables
    reference_table_stu = createReferenceTable(pseudo_labels_stu)
    # reference_table_sup = createReferenceTable(pseudo_labels_sup)

    # Display reference tables
    # display(reference_table_stu)
    # display(reference_table_sup)

    # Create stacked bar charts with pseudo labels
    uvals = [i for i in uniquevals if i in evalcountDfStu.columns]
    figstu = createStackedBarChart(evalcountDfStu[['PseudoLabel'] + uvals], 'Student General Evaluation', 'PseudoLabel')
    figsup = createStackedBarChart(evalcountDfSup[['PseudoLabel'] + uvals], 'Supervisor General Evaluation', 'PseudoLabel')

    # figstu = createStackedBarChart(evalcountDfStu, 'Student General Evaluation Counts', 'PseudoLabel')
    # figsup = createStackedBarChart(evalcountDfSup, 'Supervisor General Evaluation Counts', 'PseudoLabel')
    # Convert both figures to images
    imgStu = figToImage(figstu)
    imgSup = figToImage(figsup)
    # Create a new figure with 1 row and 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(figSize[0], figSize[1]))
    # Display images in the subplots
    axes[0].imshow(imgStu)
    axes[0].axis('off')
    axes[1].imshow(imgSup)
    axes[1].axis('off')
    plt.tight_layout()
    evalImgOp = addPlotImage(fig, 0.9)
    refTable = createTable(reference_table_stu, 'Reference Table', colRatio = [4, 1], customTextCols=[0])
    return evalImgOp, refTable

In [ ]:


def createGeneralReport(df, cohort, savefolder):
    doc = SimpleDocTemplate(f'{savefolder}\\{cohort} Summary.pdf', pagesize=pageSize, leftMargin=leftMargin,
                            rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    idCounts = df[colId].value_counts()
    ids = idCounts.index
    elements = []
    patientCountDf = pd.DataFrame(columns=['Student ID', 'Student Name', '# Patients', 'Patient Ages',
                                           '#18+', '#7-17', '#0-6', 'Clinic'])
    evalDf = pd.DataFrame(columns=['Student ID', 'Student Name', '# Yes', '# No', '# N/A', " No%"])
    
    for id in ids:
        studentDf = df[df[colId] == id]
        studentName = studentDf['given name'].values[0] + ' ' + studentDf['family name'].values[0]
        studentDf['Patient Count'] = studentDf[colNPts.lower()].apply(getNPatients)
        studentDf['Patient Count'] = studentDf['Patient Count'].replace('Patient FTA', 0).astype(int)
        patientCount = studentDf['Patient Count'].sum()
        ageList = []
        for col in colAge:
            studentDf[col] = studentDf[col].astype(float)
            ageList.extend(studentDf[col].dropna().to_list())
        ageList.sort()
        n18plus = len([age for age in ageList if age >= 18])
        n7to17 = len([age for age in ageList if age >= 7 and age < 18])
        n0to6 = len([age for age in ageList if age >= 0 and age < 7])
        clinic = studentDf['clinic'].values[0]
        patientCountDf = pd.concat([patientCountDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName], '# Patients': [patientCount],
             "Patient Ages": [ageList], '#18+': [n18plus], '#7-17': [n7to17], '#0-6': [n0to6], 'Clinic': [clinic]})])
    
        # get the evaluation counts for supervisorcol
        nYes = 0
        nNo = 0
        nNA = 0
        
        for col in supervisorcol:
            nYes += studentDf[col].str.contains('Yes').sum()
            nNo += studentDf[col].str.contains('No').sum()
            nNA += studentDf[col].str.contains('NA').sum()
        nopercent = nNo / (nYes + nNo + nNA + 0.001) * 100
        evalDf = pd.concat([evalDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName],
                                                  '# Yes': [nYes], '# No': [nNo], '# N/A': [nNA], ' No%': [nopercent]})])
    
    evalDf.sort_values(by=[' No%', '# Yes'], ascending=[True, False], inplace=True)
    
    # Create a stacked plot of evaluation counts with student id as x axis
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]))
    evalDf.set_index('Student ID')[['# Yes', '# No', '# N/A']].iloc[::-1].plot(kind='barh', stacked=True, ax=ax, width=0.8)
    ax.set_xlabel('Count')
    ax.set_ylabel('Student ID')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=8)
    plt.subplots_adjust(hspace=0.5)
    plt.tight_layout()
    img = addPlotImage(fig, 0.9)
    
    elements.append(Paragraph(f'Performance Overview for {cohort}', subsubheadingStyle))
    elements.append(Spacer(1, 12))
    elements.append(img)
    elements.append(Spacer(1, 6))
    elements.append(Paragraph("The students at the bottom are with highest % of No Responses", tableTextStyle))
    
    # Create a table for patientCountDf with bar graphs in the Patient Ages column
    table_data = [['Student ID', 'Student Name', '# Patients', 'Patient Ages', '#18+', '#7-17', '#0-6']]
    patientCountDf.sort_values(by='# Patients', ascending=False, inplace=True)
    

    unique_clinics = patientCountDf['Clinic'].unique()
    print(unique_clinics)
    clinic_colors = {clinic: color for clinic, color in zip(unique_clinics, sns.color_palette("husl", len(unique_clinics)))}
    
    # Create a legend for clinic colors
    legend_data = [[Paragraph('Clinic', tableTextStyle), Paragraph('Color', tableTextStyle)]]
    for clinic, color in clinic_colors.items():
        legend_data.append([Paragraph(clinic, tableTextStyle), ''])
    
    legend_table = Table(legend_data, colWidths=[2 * inch, 1 * inch])
    legend_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 12),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    
    for i, (clinic, color) in enumerate(clinic_colors.items(), start=1):
        legend_table.setStyle(TableStyle([
            ('BACKGROUND', (1, i), (1, i), color)
        ]))
    

    
    for i, row in patientCountDf.iterrows():
        # Create a tiny bar graph for the Patient Ages column
        fig, ax = plt.subplots(figsize=(2, 0.5))
        sns.histplot(row['Patient Ages'], bins=[i for i in range(0, 100, 6)], ax=ax)
        ax.yaxis.set_visible(False)
        ax.xaxis.set_visible(False)
        ax.axvline(x=6.5, color='black', linestyle='dotted')
        ax.axvline(x=18, color='black', linestyle='dotted')
        plt.tight_layout()
        imgdata = io.BytesIO()
        fig.savefig(imgdata, format='png', bbox_inches='tight')
        plt.close(fig)
        imgdata.seek(0)
        img = Image(imgdata, width=4 * inch, height=1 * inch)
        
        table_data.append([row['Student ID'], Paragraph(row['Student Name'], tableTextStyle),
                           row['# Patients'], img, row['#18+'], row['#7-17'], row['#0-6']])
        
    table = Table(table_data, colWidths=[1.2 * inch, 1.5 * inch, 1 * inch, 4 * inch, 0.8 * inch, 0.8 * inch, 0.8 * inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 0.2 * inch),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
        ('BOTTOMPADDING', (3, 1), (3, -1), 0),  
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    
    for i, row in enumerate(patientCountDf.iterrows(), start=1):
        clinic = row[1]['Clinic']
        if clinic in clinic_colors:
            table.setStyle(TableStyle([
                ('BACKGROUND', (0, i), (-1, i), clinic_colors[clinic])
            ]))
    elements.append(PageBreak())
    elements.append(Paragraph(f'Patient Count and Age Distribution for {cohort}', subsubheadingStyle))
    elements.append(Spacer(1, 12))
    text = f"""
The table shows ages of patients with patient count and age group distribution, but all information
                              regarding patient ages was not filled appropriately. Thus, the age distribution is not accurate for
                              all students.The dotted lines in the graphs indicate the age groups 0-6 and 7-17.
    """
    text2 = f"""Average Patient count is {round(patientCountDf['# Patients'].mean(), -1)} per student. """
    text3 = f"""The highest number of patients seen by a student is {patientCountDf['# Patients'].max()}."""
    text4 = f"""The lowest number of patients seen by a student is {patientCountDf['# Patients'].min()}.""" 
    text5 = "Data is not very accurate due to missing fields in the forms but should be close."
    elements.append(Paragraph(text, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text2, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text3, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text4, tableTextStyle))
    elements.append(Spacer(1, 6))
    elements.append(Paragraph(text5, tableTextStyle))
    elements.append(Spacer(1, 12))

    elements.append(Paragraph('Clinic Colors Legend', subsubheadingStyle))
    elements.append(Spacer(1, 12))
    elements.append(legend_table)
    elements.append(Spacer(1, 12))

    elements.append(table)
    
    # doc.build(elements)
    # display(patientCountDf)
    # display(evalDf)
    return elements
# savefolder = f'{folder}'
# createGeneralReport(dfboh3, 'BOH3', savefolder)

In [ ]:
print(df[colId.lower()].value_counts())
print(df[colId.lower()].nunique())

In [ ]:
plt.ioff()
# get the ages from all the columns
def plotages(df, title):
    ages = []
    for col in colAge:
        ages.extend(df[col].to_list())
    
    # remove nan
    ages = [age for age in ages if not np.isnan(age)]
    print(ages)
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]*0.8))
    sns.histplot(ages, bins=[i for i in range(0, 100, 10)], ax=ax)
    # show total number of ages
    total = len(ages)
    plt.text(0.90, 0.90, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    plt.title(title)
    plt.xlabel('Age')
    return fig

# function to plot pie of counts of column
def plotpiecounts(df, col, title, ax):
    counts = df[col].value_counts()
    print(counts)
    total = counts.sum()
    counts.plot.pie(autopct=lambda x: autopct(x, total), ax=ax,  textprops={'fontsize': 16})
    ax.axis('off')
    plt.text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes,
             fontsize=16)
    ax.set_title(title)

def plotpie(df, title):
    # plot pie chart of counts of Role column
    fig, ax = plt.subplots(figsize=figSize)
    total = len(df)
    df[colRole.lower()].value_counts().plot.pie(ax=ax, autopct=lambda x: autopct(x, total))
    plt.title(title)
    plt.text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    return fig



def plotagegroups(df, title):
    dummydf = pd.DataFrame(columns = ['Age Group', 'Code'])
    for ageCol, codeCol in zip(colAgeNew, colCodeNew):
        dummydf = pd.concat([dummydf, df[[ageCol, codeCol]].rename(columns={ageCol: 'Age Group', codeCol: 'Code'})])
    # display(dummydf)
    # remove nan ages
    dummydf = dummydf[dummydf['Age Group'].notna()]
    if dummydf.shape[0] == 0:
        return None
    print(dummydf['Age Group'].unique())
    # display(dummydf.head(20))
    nsubplots = len(dummydf['Age Group'].unique())
    # create a square grid of subplots
    rows = int(np.ceil(nsubplots/2))
    cols = 2
    fig, ax = plt.subplots(nsubplots, 1, figsize=(figSize[0], figSize[1]))    
    # ax = ax.flatten()
    j=0
    removegraph = []
    for i, group in enumerate(dummydf['Age Group'].unique()):
        axi = ax[i] if nsubplots > 1 else ax
        # remove nan entries
        print(f'{group}: {dummydf[dummydf["Age Group"]==group].shape[0]}')
        total = dummydf[dummydf['Age Group']==group].shape[0]
        # get all the codes
        codes = []
        for j, row in dummydf[dummydf['Age Group']==group].iterrows():
            codes.extend(row['Code'])
        # remove nan
        codes = [code for code in codes if code != 'nan']
        # remove if not a number
        # codes = [code for code in codes if code.isnumeric()]
        # remove if more than 5 characters
        codes = [code for code in codes if len(code) <= 5]
        print(codes)
        if len(codes) == 0:
            removegraph.append(i)
            continue
        # Calculate frequency of each code
        code_counts = pd.Series(codes).value_counts(ascending=False)
        # remove N/A n/a Nil '' indexes if they exist
        code_counts = code_counts.drop(['N/A', 'n/a', 'Nil', 'nil', ''], errors='ignore')
        # remove counts less than 2
        # if len(code_counts)>15:
        #     code_counts = code_counts[code_counts>=3]
        sns.barplot(x=code_counts.index, y=code_counts.values, ax=axi)
        axi.set_title(f'({group} years)')
        axi.set_xlabel('Items')
        axi.set_ylabel('Frequency')
        axi.set_xticklabels(axi.get_xticklabels(), rotation=90, fontsize=8)
        axi.text(0.9, 0.9, f'Total: {total}', horizontalalignment='center', verticalalignment='center', transform=axi.transAxes)
        # y ticks should be integers
        axi.yaxis.get_major_locator().set_params(integer=True)
    plt.tight_layout()
    plt.suptitle(f'{title} - #Items performed by Age Group', fontsize=16, y=1.02)
    # remove empty graphs
    print(f'Number of subplots: {nsubplots}')
    print(f'Removing graphs: {removegraph}')
    if len(removegraph):
        if nsubplots == 1:
            fig.delaxes(ax)
        else:
            for i in removegraph:
                fig.delaxes(ax[i])
    return fig
# create student, total, roles df===================================================================================
def createRoleTable(df, roleCol, studentIdCol, nameGCol, nameFCol):
    rolecountDf = pd.DataFrame(columns = ['Student ID', 'Student Name', 'Total Forms'] + df[roleCol].unique().tolist()[::-1])
    # rolecountDf = pd.DataFrame(columns = ['Student ID', 'Student Name', 'Total Forms'] )
    idList = df[studentIdCol].unique()
    for id in idList:
        studentDf = df[df[studentIdCol]==id]
        studentName = studentDf[nameGCol].values[0] + ' ' + studentDf[nameFCol].values[0]
        totalforms = len(studentDf)
        rolecount = studentDf[roleCol].value_counts().astype(int)
        rolecountDf = pd.concat([rolecountDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName],
                                                            'Total Forms': [totalforms], **rolecount.to_dict()})])
    rolecountDf.fillna(0, inplace=True)
    # sort by operator
    rolecountDf.sort_values(by='Total Forms', ascending=False, inplace=True)
    display(rolecountDf.head())
    # add Paragraph to column names
    rolecountDf.columns = [Paragraph(col, tableTextStyle) for col in rolecountDf.columns]
    rolecounttable = createTable(rolecountDf, 'Role Counts', colRatio = [1, 2.5, 1] + [1]*len(df[roleCol].unique()), customTextCols=[1],
                                bottomPadding=8)
    return rolecounttable


def createRoleBarChart(df, roleCol, studentIdCol, nameGCol, nameFCol):
    rolecountDf = pd.DataFrame(columns=['Student ID', 'Student Name', 'Total Forms'] + df[roleCol].unique().tolist()[::-1])
    idList = df[studentIdCol].unique()
    for id in idList:
        studentDf = df[df[studentIdCol] == id]
        studentName = studentDf[nameGCol].values[0] + ' ' + studentDf[nameFCol].values[0]
        totalforms = len(studentDf)
        rolecount = studentDf[roleCol].value_counts().astype(int)
        rolecountDf = pd.concat([rolecountDf, pd.DataFrame({'Student ID': [id], 'Student Name': [studentName],
                                                            'Total Forms': [totalforms], **rolecount.to_dict()})])
    rolecountDf.fillna(0, inplace=True)
    rolecountDf.set_index('Student Name', inplace=True)
    rolecountDf.sort_values(by='Total Forms', ascending=True, inplace=True)
    
    fig, ax = plt.subplots(figsize=(figSize[0]/3, figSize[1]/2))
    # rearrange the columns to be Operator, SO, Assisstant, Observation
    allrolecols = ['Operator', 'SO', 'Assistant', 'Observation']
    thisallcols = [col for col in allrolecols if col in rolecountDf.columns]
    rolecountDf = rolecountDf[['Student ID'] + thisallcols]
    rolecountDf.set_index('Student ID', inplace=True)
    rolecountDf.plot(kind='barh', stacked=True, ax=ax, width=0.8)
    ax.set_xlabel('Count')
    ax.set_ylabel('Student ID')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=8)    
    ax.set_title('Role Counts by Student')
    plt.tight_layout()
    return fig

def createcohortreport(df, cohort):

    dfoperator = df[df[colRole.lower()] == 'Operator']
    elements = []

    fig, ax = plt.subplots(1, 1, figsize=(figSize[0], figSize[1]))
    plotpiecounts(df, colRotation, None, ax)
    imgrotation = addPlotImage(fig, 0.5)
    if cohort == 'All':
        elements.append(KeepTogether([
            Paragraph('Clinic Distribution', subsubheadingStyle),
            Spacer(1, 12),
            imgrotation
        ]))
        elements.append(Spacer(1, 24))
    elements.append(KeepTogether([
        Paragraph('Role Distribution', subsubheadingStyle),
        Spacer(1, 12),
        addPlotImage(plotpie(df, None), 0.6)
    ]))
    elements.append(Spacer(1, 24))
    # plot age distribution
    fig2 = plotages(df, f'Age distribution for {cohort}')
    elements.append(KeepTogether([
        Paragraph('Age Distribution', subsubheadingStyle),
        Spacer(1, 12),
        addPlotImage(fig2, 0.6)
    ]))
    # plot number of patients FTAs
    elements.append(Spacer(1, 24))
    fig3, ax3 = plt.subplots(1, 1, figsize=(figSize[0], figSize[1]))
    plotpiecounts(df[df[colRole.lower()]=='Operator'], colNPtsNumber, None, ax3)
    img3 = addPlotImage(fig3, 0.6)
    elements.append(KeepTogether([
        Paragraph('Patient Count Distribution', subsubheadingStyle),
        Spacer(1, 12),
        img3
    ]))

    # fig3 = plotagegroups(dfoperator, f'{cohort}')
    # if fig3:
    #     img3 = addPlotImage(fig3, 0.85)
    #     elements.append(img3)
    # elements.append(PageBreak())
    # elements.extend(createGeneralReport(df, cohort, folder))
    # doc.build(elements)
    # stacked bar plot of evaluation counts

    evalcountDf = getEvalCounts(dfoperator, allstudentcol)
    # evalcountDf.sort_values(by=' No%', ascending=True, inplace=True)
    # fig, ax = plt.subplots(figsize=(figSize[0], figSize[1]))
    # evalcountDf.set_index('Question')[['Yes', 'No']].iloc[::-1].plot(kind='barh', stacked=True, ax=ax, width=0.8)
    # ax.set_xlabel('Count')
    # ax.set_ylabel('Question')
    # ax.set_yticklabels(ax.get_yticklabels(), fontsize=8)
    # plt.tight_layout()

    if len(dfoperator) == 0:
        elements.append(Paragraph('No Operator Forms', tableTextStyle))
    else:
        img, refTable = createEvalPlots(dfoperator, studentcol, supervisorcol)
        elements.append(KeepTogether([(Paragraph('Evaluation Counts', subsubheadingStyle)),
        Spacer(1, 12),
            img,
            Spacer(1, 12),
            refTable
        ]))
        dfoperator.replace({'Not assessed': 'NA', 'Not relevant': 'NA'}, inplace=True)
        img2, refTable2 = createEvalPlots(dfoperator, otherstudentcol, othersupervisorcol)
        elements.append(KeepTogether([(Paragraph('Evaluation Counts Other', subsubheadingStyle)),
        Spacer(1, 12),
            img2,
            Spacer(1, 12),
            Spacer(1, 24),
            Paragraph('There is still the problem with NA, a lot of items are not assessed which means. We should make dynamic checklists', tableTextStyle),
            Spacer(1, 24),
            refTable2
        ]))
        # clinical incident table
        display(dfoperator.head())
        dfCI = dfoperator[dfoperator[colCI.lower()] == 'Yes']
        dfCI['Student Name'] = dfCI[colGivenName] + ' ' + dfCI[colFamilyName]
        dfCI.rename(columns={colCIExp.lower(): colCIExp}, inplace=True)
        dfCI.rename(columns={colId.lower(): colId.upper()}, inplace=True)   
        dfCI = dfCI[[colId.upper(), 'Student Name', colCIExp]]
        elements.append(Spacer(1, 24))
        if len(dfCI) > 0:
            elements.append(KeepTogether([
                Paragraph('Clinical Incidents', subsubheadingStyle),
                Spacer(1, 12),
                createTable(dfCI, 'Clinical Incidents', colRatio=[1, 2, 2], customTextCols=[1, 2])
            ]))
        else:
            elements.append(KeepTogether([
                Paragraph('Clinical Incidents', subsubheadingStyle),
                Spacer(1, 12),
                Paragraph('No Clinical Incidents', tableTextStyle)
            ]))
        elements.append(PageBreak())
    # elements.append(Paragraph('Role Distribution', subsubheadingStyle))
    # elements.append(Spacer(1, 12))
    elements.append(createRoleTable(df, colRole.lower(), colId, colGivenName, colFamilyName))
    
    # elements.append(PageBreak())
    # elements.append(Paragraph('Role Distribution Bar Chart', subsubheadingStyle))
    # elements.append(Spacer(1, 12))
    # elements.append(addPlotImage(createRoleBarChart(df, colRole.lower(), colId, colGivenName, colFamilyName), 0.9))
    
    return elements

In [ ]:
def createfronttable(df: pd.DataFrame, title = 'Front Page Summary', type_=None):
    nforms = len(df)
    # age range  dictionary
    ageDict = df[colAgeNew].apply(pd.Series.value_counts).to_dict()
    result = {}
    for group in ageDict.values():
        for key, value in group.items():
            result[key] = result.get(key, 0) + (value if not np.isnan(value) else 0)
    ageDict = result
    print(ageDict)
    # total patients
    df['num patients'] = df[colNPts.lower()].apply(getNPatients) 
    df['num patients'] = df['num patients'].replace('Patient FTA', 0).astype(int)
    # get total patients seen by each student
    studentpatients = df.groupby(colId)['num patients'].sum().to_dict()
    print(studentpatients)
    # filter out max and min vlaues
    maxPatients = max(studentpatients.values())
    minPatients = min(studentpatients.values())
    print(minPatients, maxPatients)
    # get num patients for each clinic
    clinicPatients = df.groupby('clinic')['num patients'].sum().to_dict()
    clinicPatietnsText = "<br/>".join([f"{key}: {value}" for key, value in clinicPatients.items()])
    totalPatients = df['num patients'].sum()
    print(totalPatients)
    # average patient per student
    dfoperators = df[df[colRole.lower()] == 'Operator']
    noperators = len(dfoperators)
    noperatorstudents = len(dfoperators[colId].unique())
    nstudents = len(df[colId].unique())
    avgPatients = totalPatients / noperatorstudents
    # average patient per form
    avgPatientsForm = totalPatients / noperators
    print(avgPatients)
    print(avgPatientsForm)
    # priority counts
    priorityDict = df[colPriority].apply(pd.Series.value_counts).to_dict()
    result = {}
    for group in priorityDict.values():
        for key, value in group.items():
            # if , then split the key and add the value to the key
            if ',' in key:
                keys = key.split(',')
                for key in keys:
                    result[key] = result.get(key, 0) + (value if not np.isnan(value) else 0)
            else:
                result[key] = result.get(key, 0) + (value if not np.isnan(value) else 0)
    priorityDict = result
    print(priorityDict)
    # item codes count
    itemCodeList = []
    for col in colCodeNew:
        itemCodeList.extend(df[col].to_list())
    itemCodeList = [item for sublist in itemCodeList for item in sublist]
    itemCodeDict = {item: itemCodeList.count(item) for item in set(itemCodeList)}
    # remove nan
    itemCodeDict.pop('nan', None)
    print(itemCodeDict)
    # get top 5 item codes
    top5ItemCodes = dict(sorted(itemCodeDict.items(), key=lambda x: x[1], reverse=True)[:5])
    print(top5ItemCodes)
    # entrustment counts
    entrustmentCounts = df['entrustment'].value_counts().to_dict()
    # split the keys with :
    entrustmentCounts = {key.split(':')[0]: value for key, value in entrustmentCounts.items()}
    print(entrustmentCounts)
    # extract number Lvl \d+ from entrustment
    df['entrustment lvl'] = df['entrustment'].astype(str).str.extract(r'Lvl (\d+)').fillna(0)
    df['entrustment lvl'] = df['entrustment lvl'].astype(int)
    # yes/no/na counts
    df['nYesStudent'] = df[studentcol].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['nNoStudent'] = df[studentcol].apply(lambda x: (x == 'No').sum(), axis=1)
    df['nNAStudent'] = df[studentcol].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['scoreStu'] = df['nYesStudent']/(df['nYesStudent'] + df['nNoStudent'] + 0.0001)
    df['nYesSupervisor'] = df[supervisorcol].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['nNoSupervisor'] = df[supervisorcol].apply(lambda x: (x == 'No').sum(), axis=1)
    df['nNASupervisor'] = df[supervisorcol].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['scoreSup'] = df['nYesSupervisor']/(df['nYesSupervisor'] + df['nNoSupervisor'] + 0.0001)
    df['scorediff'] = df['scoreStu'] - df['scoreSup']
    scorediffavg = df['scorediff'].mean()
    # filter out df with entrustment non zero
    dfentrustment = df[df['entrustment lvl'] > 0]
    # correlation between entrustment and score supervisor
    entrustmentCorr = dfentrustment['entrustment lvl'].corr(dfentrustment['scoreSup'])
    print(entrustmentCorr)
    # get column wise Yes/No/NA counts for supervisorcols
    dfcolumnwise = pd.DataFrame(index=supervisorcol, columns=['key', 'Question','Yes', 'No', 'NA'])
    for col in supervisorcol:
        dfcolumnwise.loc[col, 'Yes'] = df[col].str.contains('Yes').sum()
        dfcolumnwise.loc[col, 'No'] = df[col].str.contains('No').sum()
        dfcolumnwise.loc[col, 'NA'] = df[col].str.contains('NA').sum()
        dfcolumnwise.loc[col, 'Question'] = fullnamedict[col].split(' - ')[-1].split('\n')[-1]
        dfcolumnwise.loc[col, 'key'] = fullnamedict[col].split(' - ')[-1].split('\n')[0]
    dfcolumnwise['score'] = dfcolumnwise['Yes']/(dfcolumnwise['Yes'] + dfcolumnwise['No'] + 0.0001)
    dfcolumnwise.sort_values(by='score', ascending=False, inplace=True)
    bestQuestion = dfcolumnwise.loc[dfcolumnwise.index[0], 'key']
    best3Questions = dfcolumnwise.loc[dfcolumnwise.index[:2], 'key']

    worstQuestion = dfcolumnwise.loc[dfcolumnwise.index[-1], 'key']
    worst3Questions = dfcolumnwise.loc[dfcolumnwise.index[-3:], 'key']

    # get role wise counts
    dfrole = df[colRole.lower()].value_counts().to_dict()
    roltext = "<br/>".join([f"{key}: {value}" for key, value in dfrole.items()])
    # display(dfcolumnwise)
    
    ageOrder = ["0-6", "7-17", "18+"]

    # Convert ageDict to formatted text in the specified order
    agetext = "<br/>".join([f"{key}: {int(ageDict[key])}" for key in ageOrder if key in ageDict])
    prioritytext = "<br/>".join([f"{key.split('Group ')[-1]}: {int(priorityDict[key])}" for key in priorityDict])
    # create a table for the front page
    dfFront = pd.DataFrame([
    ['Total number of forms submitted', nforms],
    ['Role Distribution', roltext],
    ['Age Range Counts', agetext],
    ['Total number of patients managed by students', totalPatients],
    ['Average number of patients managed per student (min - max)', f'{round(avgPatients, 2) if not pd.isna(avgPatients) else 0}   ({minPatients} - {maxPatients})'],
    ['Number of forms receiving entrustment level 3 or 4', (df['entrustment lvl'] >= 3).sum()],
    ['Number of forms receiving entrustment level 2', (df['entrustment lvl'] == 2).sum()],
    ['Number of forms receiving entrustment level 1', (df['entrustment lvl'] == 1).sum()],
    ['Correlation between entrustment and supervisor score(0-1)<br/>(Higher correlation implies supervisor are giving entrustment scores based on objective checklist scores and have less latent bias, should be 0.2+ <br/>*Not valid for small number of students)', round(entrustmentCorr, 2)],
    ['Difference in student and supervisor ratings (avg)', round(scorediffavg,3)],
    ['Top 5 Item Codes', ', '.join(top5ItemCodes.keys())],  
    ['Priority Group', prioritytext],
    ['Clinic Patient Counts', clinicPatietnsText],
    ['Areas students are performing well in', ", ".join(best3Questions)],
    ['Areas that need improvement', ", ".join(worst3Questions)],
], columns=['Metric', 'Value'])
    # if type_ == 'clinic': # remove last row
    #     dfFront = dfFront.iloc[:-1]
    # # Adding Age Range
    # for key, value in ageDict.items():
    #     dfFront.loc[len(dfFront)] = [f'Age range {key}', value]

    # Adding Priority Group Counts
    # for key, value in priorityDict.items():
    #     dfFront.loc[len(dfFront)] = [f'Priority Group {key}', int(value)]

    # Adding Top 5 Item Codes
    # for key, value in top5ItemCodes.items():
    #     dfFront.loc[len(dfFront)] = [f'Item Code {key}', value]
    # display(dfFront)
    display(dfFront)
    dfFrontTable = createTable(dfFront, title, colRatio=[3,2], customTextCols=[0, 1], tableTextStyle=tableTextStyleSmall)
    # table is 2nd last item of keepTogether dfFrontTable
    table = dfFrontTable._content[2]
    # change top and bottom padding of the table
    padding = 12
    if type_ == 'clinic':
        padding = 6
    table.setStyle(TableStyle([
        ('TOPPADDING', (0, 0), (-1, -1), padding),
        ('BOTTOMPADDING', (0, 0), (-1, -1), padding),
    ]))
    # get all the contents of dfFrontTable
    # for content in dfFrontTable._content:
    #     print(content)
    # change text of 3rd row and 2nd column to age paragraph
    numRows = len(table._cellvalues)  # Number of rows
    numCols = len(table._cellvalues[0]) if numRows > 0 else 0  # Number of columns

    print(f"Rows: {numRows}, Columns: {numCols}")
    # table._cellvalues[2][1] = ageparagraph
    return dfFrontTable
# createfronttable(dfdds4)

In [ ]:
# create reports for each rotation
for rotation in df[colRotation].unique():
    dfrot = df[df[colRotation] == rotation]
    doc = SimpleDocTemplate(f'{folder}\\DDS3 Clinical {rotation}.pdf', pagesize= pageSize, leftMargin = leftMargin,
                            rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    elements = []
    elements.append(Paragraph(f'{rotation} Summary', headingStyle))
    elements.append(Spacer(1, 24))
    fronttable = createfronttable(dfrot, f'{rotation} Summary', 'clinic')
    elements.append(fronttable)
    elements.extend(createcohortreport(dfrot, rotation))
    doc.build(elements)

# create reports for all combined
# generalreportelements = createcohortreport(df, 'All')
# doc = SimpleDocTemplate(f'{folder}\\DDS3 Clinical All.pdf', pagesize= pageSize, leftMargin = leftMargin,
#                         rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
# elements = []
# elements.append(Paragraph('DDS3 Specialist Summary', headingStyle))
# elements.append(Spacer(1, 24))
# fronttable = createfronttable(df)
# elements.append(fronttable)
# elements.extend(generalreportelements)
# doc.build(elements)

### Student Reports

In [ ]:
# def createEvalDf(row, studentcol, supervisorcol):
#     evaldf = pd.DataFrame(columns = ['Key Competency', 'Marking Checklist', 'Student', 'Educator'])
#     for stucol, educol in zip(studentcol, supervisorcol):
#         # get common text in fullnamedict[stucol] and fullnamedict[educol]
#         # commonText = 
#         markingchecklist = fullnamedict[stucol].split(' Student Evaluation - ')[1]
#         key = ''.join(markingchecklist.split('\n')[:-1])
#         markingchecklist = markingchecklist.split('\n')[-1]
#         print(key, markingchecklist)
#         student = row[stucol]
#         educator = row[educol]
#         evaldf = pd.concat([evaldf, pd.DataFrame({'Key Competency':key, 'Marking Checklist': [markingchecklist], 
#                                                   'Student': [student], 'Educator': [educator]})])
#     return evaldf



# def createReport(studentDf, id, name, savefolder):
#     os.makedirs(savefolder, exist_ok=True)
#     print(id, name)
#     elements = []
#     doc = SimpleDocTemplate(f'{savefolder}\\{name} ({id}).pdf', pagesize= pageSize, leftMargin = leftMargin,
#                             rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
#     elements.append(Paragraph(f'{name} ({id})', headingStyle))
#     elements.append(Spacer(1, 24))
#     dfDateRoleItems = pd.DataFrame(columns=['Date', 'Role', 'Items', 'No. of Patients'])
#     for i, row in studentDf.iterrows():
#         date = row['date'].strftime('%d-%m-%Y')
#         role = row['role']
#         items = []
#         for col in colCodeNew:
#             items.extend(row[col])
#         items = [item for item in items if item != 'nan']
#         items = list(set(items))    
#         # convert to string
#         items = ', '.join(items).strip(',')
#         # get the number of patients
#         npts = getNPatients(row[colNPts.lower()])
#         # get the clinical incident
#         dfDateRoleItems = pd.concat([dfDateRoleItems, pd.DataFrame({'Date': [date], 'Role': [role], 'Items': [items], 'No. of Patients': [npts]})])
#     display(dfDateRoleItems)
#     table = createTable(dfDateRoleItems, 'Item Summary', colRatio = [1, 1.5, 3, 2], customTextCols=[2])
#     elements.append(table)
#     elements.append(PageBreak())
#     studentDf.reset_index(drop=True, inplace=True)
#     for i, row in studentDf.iterrows():
#         # for all the important columns display column name and value
#         elements.append(Paragraph(f'Form {i+1}: {row["date"].strftime("%d-%m-%Y")}', subheadingStyle))
#         elements.append(Paragraph(f'Role: {row["role"]}', subsubheadingStyle))
#         elements.append(Spacer(1, 10))
#         # fill in patient info in colPtInfo
#         lastPatient = None  # Keep track of the last patient number displayed

#         for col in colPtInfo:
#             if pd.isna(row[col.lower()]) or row[col.lower()] == 'nan':
#                 continue
            
#             # Extract patient number using regex
#             match = re.search(r'pt\.(\d+)', col.lower())
#             if match:
#                 patientNum = match.group(1)
                
#                 # Check if this is a new patient number
#                 if lastPatient != patientNum:
#                     elements.append(Paragraph(f'Patient {patientNum}', subsubheadingStyleL))  # Add bold subheading
#                     elements.append(Spacer(1, 12))  # Add some space after the heading
#                     lastPatient = patientNum  # Update the last patient tracker
            
#             # Add patient info
#             if 'code' in col.lower() or 'categories' in col.lower():
#                 continue
#             elements.append(Paragraph(f'{fullnamedict[col]}: {row[col]}', tableTextStyleL))
#             elements.append(Spacer(1, 6))
#         # fill in the important columns
#         for col in importantCols:
#             if pd.isna(row[col.lower()]) or row[col.lower()] == 'nan':
#                 continue
#             print(f'{col}: {row[col.lower()]}')
#             elements.append(Paragraph(f'{col.strip()}', subsubheadingStyleL))
#             elements.append(Spacer(1, 6))
#             elements.append(Paragraph(f'{row[col.lower()].strip()}', tableTextStyleL))
#             elements.append(Spacer(1, 12))
#         row[colNPtsNumber] = 0 if row[colNPtsNumber] == 'Patient FTA' else row[colNPtsNumber]
#         row[colNPtsNumber] = int(row[colNPtsNumber])
#         if row[colRole.lower()] == 'Operator' and row[colNPtsNumber] > 0:
#             # elements.append(Paragraph(f'Evaluation', subheadingStyle))
#             evaldf = createEvalDf(row, allstudentcol, allsupervisorcol)
#             evaldf.replace({'Not assessed': 'NA', 'Not relevant': 'NA', 'Not reviewed':'NA', 'nan': 'NA'}, inplace=True)
#             evaldf['Student'] = evaldf['Student'].astype(str)
#             evaldf['Educator'] = evaldf['Educator'].astype(str)
#             evaldf.replace({'nan': 'NA'}, inplace=True)
#             # remove rows where both student and educator are NA, nan
#             evaldf = evaldf[~((evaldf['Student']=='NA') & (evaldf['Educator']=='NA'))]
#             table = createTable(evaldf, 'Evaluation', colRatio = [2, 5, 1, 1], customTextCols=[0, 1], cellHighlight=True, 
#                                 tableTextStyle=tableTextStyleSmall)
#             elements.append(table)


#         elements.append(PageBreak())
#         # break

#     doc.build(elements)


#     pass

# # BOH3 first
# def createStudentReport(df, cohort, savefolder):
#     # print(dfboh3.columns)
#     idCounts = df[colId].value_counts()
#     ids = idCounts.index
#     print(idCounts)
#     for id in ids:
#         studentDf = df[df[colId]==id]
#         studentName = studentDf['family name'].values[0] + ' ' + studentDf['given name'].values[0]
#         # display(studentDf)
#         createReport(studentDf, id, studentName, os.path.join(savefolder, cohort))
#         # break

# savefolder = f'{folder}\\Student Reports'
# createStudentReport(df, 'DDS3', savefolder) 

    

# IRT

In [ ]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az

# Function to prepare data for IRT modeling
def prepare_irt_data(df, eval_prefix):
    # Extract binary evaluation columns
    eval_cols = [col for col in df.columns if eval_prefix in col]
    # Convert to numpy array for modeling
    response_matrix = df[eval_cols].fillna(0).values.astype(int)
    return response_matrix

# Prepare datasets for both cohorts and both evaluation sources
irt_data = {
    "BOH3_Student": prepare_irt_data(boh3_student, 'Evaluation#1'),
    "BOH3_Supervisor": prepare_irt_data(boh3_supervisor, 'Evaluation#2'),
    "DDS4_Student": prepare_irt_data(dds4_student, 'Evaluation#1'),
    "DDS4_Supervisor": prepare_irt_data(dds4_supervisor, 'Evaluation#2')
}

# Function to fit IRT models: 1PL, 2PL, and 3PL
def fit_irt_models(data, model_type="1PL"):
    n_students, n_items = data.shape
    
    with pm.Model() as model:
        # Priors for item parameters
        difficulty = pm.Normal("difficulty", mu=0, sigma=2, shape=n_items)
        
        if model_type in ["2PL", "3PL"]:
            discrimination = pm.HalfNormal("discrimination", sigma=2, shape=n_items)
        else:
            discrimination = 1  # Fixed for 1PL
        
        if model_type == "3PL":
            guessing = pm.Beta("guessing", alpha=5, beta=20, shape=n_items)
        else:
            guessing = 0  # Fixed for 1PL and 2PL
        
        # Latent ability for each student
        ability = pm.Normal("ability", mu=0, sigma=1, shape=n_students)
        
        # Probability of correct response (logistic IRT model)
        p = pm.math.sigmoid(discrimination * (ability[:, None] - difficulty) + guessing)
        
        # Likelihood
        responses = pm.Bernoulli("responses", p, observed=data)
        
        # Sample the posterior
        trace = pm.sample(1000, tune=1000, cores=4, return_inferencedata=True)
        
    return trace

# Fit 1PL, 2PL, and 3PL models for each dataset
irt_results = {}

for dataset_name, matrix in irt_data.items():
    irt_results[dataset_name] = {
        "1PL": fit_irt_models(matrix, "1PL"),
        "2PL": fit_irt_models(matrix, "2PL"),
        "3PL": fit_irt_models(matrix, "3PL"),
    }

# Summarize results for each model
summary_results = {}

for dataset_name, models in irt_results.items():
    summary_results[dataset_name] = {
        "1PL": az.summary(models["1PL"]),
        "2PL": az.summary(models["2PL"]),
        "3PL": az.summary(models["3PL"]),
    }

# Display summary results for analysis
summary_results
